# TRATAMIENTO DE DATOS CON PYSPARK

In [17]:
# 1. CONFIGURACIÓN E IMPORTACIONES

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline



# Iniciamos Spark
spark = SparkSession.builder \
    .appName("FinPlus_ETL_Limpieza") \
    .getOrCreate()

# Rutas de los datos
DATA_PATH = "/home/jovyan/work/data/"  
OUTPUT_PATH = "/home/jovyan/work/data/curated/"

In [18]:
# 2. INGESTA DE DATOS 

# Cargar CLIENTS.csv
# Usamos header=True para leer la cabecera e inferSchema=True para detectar números
df_clients = spark.read.csv(DATA_PATH + "CLIENTS.csv", header=True, inferSchema=True, sep=',')

# Cargar BEHAVIOURAL.parquet
df_behav = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")

# Verificamos qué columnas tenemos 
print("Esquema Clients:")
df_clients.printSchema()

print("Esquema Behavioural:")
df_behav.printSchema()

Esquema Clients:
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = true)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- CAR_AGE: double (nullable = true)
 |-- FAMILY_SIZE: double (nullable = true)
 |-- REACTIVE_SCORING: double (nullable = true)
 |-- PROACTIVE_SCORING: double (nullable = true)
 |-- BEHAVIORAL_SCORING: double (nullable = true)
 |-- DAYS_LAST_INFO_CHANGE: d

Primero analizamos las variables de tipo string. Hay de distintos tipos:

- Categóricas puras: "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY" (en ambas tablas).

- Categóricas numéricas: "NON_COMPLIANT_CONTRACT", "DIGITAL_CLIENT".

- Fecha: "DATE" (sólo en behavioural).

- Identificadores: "CLIENT_ID" (en ambas tablas), "CONTRACT_ID".

Analizamos las filas duplicadas

In [19]:
def auditar_duplicados_completo(df, nombre_tabla):
    print(f"VERIFICACIÓN DE DUPLICADOS: {nombre_tabla}\n ")
    
    # 1. Cálculos Básicos
    total_rows = df.count()
    distinct_rows = df.distinct().count()
    num_duplicados = total_rows - distinct_rows
    
    print(f"• Total filas:      {total_rows}")
    print(f"• Filas únicas:     {distinct_rows}")
    print(f"• Duplicados:       {num_duplicados}")
    
    # 2. Lógica Condicional
    if num_duplicados > 0:
        pct = (num_duplicados / total_rows) * 100
        print(f"\n AVISO: Hay {num_duplicados} filas repetidas ({pct:.2f}%).")
        print("   Mostrando ejemplos de filas idénticas:")
        
        # Esta parte solo se ejecuta si hay duplicados 
        # Agrupamos por TODAS las columnas para encontrar filas 100% idénticas
        (df.groupBy(df.columns)
           .count()
           .where(F.col("count") > 1)
           .orderBy(F.col("count").desc()) # Ponemos las más repetidas arriba
           .show(5, truncate=False))
        
    else:
        print("\nLimpio. No existen filas duplicadas exactas.")
        

# --- EJECUCIÓN ---
auditar_duplicados_completo(df_clients, "CLIENTS")
auditar_duplicados_completo(df_behav, "BEHAVIOURAL")

VERIFICACIÓN DE DUPLICADOS: CLIENTS
 
• Total filas:      162977
• Filas únicas:     162977
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.
VERIFICACIÓN DE DUPLICADOS: BEHAVIOURAL
 
• Total filas:      1724854
• Filas únicas:     1724854
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.


Ahora analizamos los posibles IDs duplicados

In [20]:
def auditar_clave_primaria(df, col_id, nombre_tabla):
    print(f"VERIFICACIÓN DE CLAVE ÚNICA ({col_id}): {nombre_tabla}\n ")
    
    # 1. Contamos filas totales
    total = df.count()
    
    # 2. Contamos IDs únicos
    unicos = df.select(col_id).distinct().count()
    
    # 3. Diferencia
    dif = total - unicos
    
    if dif > 0:
        print(f"AVISO: Hay {dif} IDs repetidos que NO son filas idénticas.")
        print("   Esto significa que tienes clientes con datos conflictivos.")
        print("   Ejemplo de IDs repetidos:")
        
        # Mostramos cuáles son los culpables
        (df.groupBy(col_id)
           .count()
           .where(F.col("count") > 1)
           .show(5))
    else:
        print(f"CORRECTO: La columna {col_id} es una clave primaria única.")

# Ejecutamos solo para CLIENTS
auditar_clave_primaria(df_clients, "CLIENT_ID", "CLIENTS")

VERIFICACIÓN DE CLAVE ÚNICA (CLIENT_ID): CLIENTS
 
CORRECTO: La columna CLIENT_ID es una clave primaria única.


Como vemos que no hay duplicados, no es necesario hacer limpieza de estos. Nos enfocaremos en los NaNs.

In [21]:
# 1. Definimos la función mejorada (le añadimos un título para que quede claro)
def inspeccionar_datos(df, nombre_tabla):
    
    print(f"INSPECCIÓN DE: {nombre_tabla}\n")
    
    print(f"Dimensiones: {df.count()} filas x {len(df.columns)} columnas")
    
    print(f"\n--- 1. Conteo de Nulos ---")
    # Calculamos nulos
    exprs_nulos = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]
    # Mostramos verticalmente para leer mejor
    df.agg(*exprs_nulos).show(vertical=True, truncate=False)

# 2. Ejecutamos la inspección 
inspeccionar_datos(df_clients, "CLIENTS")
inspeccionar_datos(df_behav, "BEHAVIOURAL")

INSPECCIÓN DE: CLIENTS

Dimensiones: 162977 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0-----------------------------
 CLIENT_ID                   | 0      
 NON_COMPLIANT_CONTRACT      | 0      
 NAME_PRODUCT_TYPE           | 0      
 GENDER                      | 0      
 TOTAL_INCOME                | 0      
 AMOUNT_PRODUCT              | 0      
 INSTALLMENT                 | 7      
 EDUCATION                   | 39640  
 MARITAL_STATUS              | 2      
 HOME_SITUATION              | 0      
 REGION_SCORE                | 0      
 AGE_IN_YEARS                | 0      
 JOB_SENIORITY               | 29174  
 HOME_SENIORITY              | 0      
 LAST_UPDATE                 | 0      
 OWN_INSURANCE_CAR           | 0      
 CAR_AGE                     | 107550 
 FAMILY_SIZE                 | 2      
 REACTIVE_SCORING            | 91901  
 PROACTIVE_SCORING           | 337    
 BEHAVIORAL_SCORING          | 32246  
 DAYS_LAST_INFO_CHANGE       | 1      
 NUMBER_OF_

Se observa que solamente tendremos que tratar nulls de la tabla de clientes. Pero además, no nos interesa eliminar filas porque eliminaríamos clientes, y menos en variables con tantos nulls como CAR_AGE, donde estaríamos eliminando más del 66% de los clientes. La estrategia ganadora en Big Data es "Imputar lo masivo, borrar lo anecdótico".

Primero de todo, eliminaremos a los clientes de los que no se tengan casi datos, pues prácticamente es como si no existiesen. Comprobamos primero si, aunque no tengamos info personal tienen movimientos.

In [22]:
# 1. Cargamos de nuevo CLIENTS original (solo para esta prueba)
df_raw = spark.read.csv(DATA_PATH + "CLIENTS.csv", header=True, inferSchema=True, sep=',')

# 2. Identificamos a los que vamos a borrar (los que tienen muchos nulos)
# thresh=25 mantenía a los buenos. Así que buscamos lo contrario.
# Para replicar la lógica inversa exacta, calculamos cuántos nulos tienen.

# Contamos cuántas columnas NO son nulas por fila
from itertools import chain
cols_check = df_raw.columns
expr = sum([F.when(F.col(c).isNotNull(), 1).otherwise(0) for c in cols_check])

# Filtramos los "Malos" (tienen menos de 25 columnas con datos)
df_zombies = df_raw.withColumn("non_nulls", expr).filter(F.col("non_nulls") < 25)

print(f"Detectados {df_zombies.count()} clientes 'Zombie' candidatos a borrar.")

# 3. CRUCE DE LA VERDAD
# Cruzamos estos Zombies con Behavioural. 
# Si sale 0, tu decisión fue perfecta. Si sale algo, cuidado.
zombies_con_dinero = df_zombies.join(df_behav, on="CLIENT_ID", how="inner")

count_risk = zombies_con_dinero.count()

if count_risk == 0:
    print(f"\n Ninguno de los clientes eliminados tenía actividad bancaria.")
else:
    print(f"\nAVISO: Hay {count_risk} clientes con pocos datos pero con movimientos.")

Detectados 1899 clientes 'Zombie' candidatos a borrar.

AVISO: Hay 6863 clientes con pocos datos pero con movimientos.


Con este resultado, esperaremos a eliminarlos después del JOIN de las tablas.

________________________________________________________________________________________________________________________

####### ELIMINAR "CLIENTES ZOMBIE" (Filas con demasiados nulos)

#######print(f"Filas antes de limpieza fina: {df_clients.count()}")

######## Tenemos unas 45 columnas. Si a un cliente le faltan más de 20 datos, no nos sirve.
######## thresh=25 significa: "Mantener solo si tiene al menos 25 columnas con datos válidos"
#######df_clients = df_clients.dropna(thresh=25) 

#######print(f"Filas tras limpieza fina: {df_clients.count()}")
________________________________________________________________________________________________________________________

In [23]:

# 3. LIMPIEZA Y TRANSFORMACIÓN

# A) LIMPIEZA DE STRINGS (CLIENTS)
# Lista de tus columnas categóricas reales (copiadas de tu esquema)
cols_categ_puras = [
    "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY"
]

# Normalizamos: quitamos espacios (trim) y pasamos a mayúsculas o minúsculas
for col_name in cols_categ_puras:
    # Solo si la columna existe en el dataframe
    if col_name in df_clients.columns:
        df_clients = df_clients.withColumn(col_name, F.trim(F.upper(F.col(col_name))))

# B) CONVERSIÓN DE FECHAS (BEHAVIOURAL)
# Tu columna DATE es string, hay que pasarla a formato fecha
# Spark suele ser listo, pero si falla, prueba con formato específico ej: "dd/MM/yyyy"
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))



In [24]:
# C) LIMPIEZA DE NULOS 

# 1. GRUPO "NO APLICA" (Rellenar con -1)
# Variables donde Nulo significa "No tiene" o "No disponible"
# CAR_AGE (66% nulos), JOB_SENIORITY, SCORING...
cols_flag = ["CAR_AGE", "JOB_SENIORITY", "REACTIVE_SCORING", "BEHAVIORAL_SCORING", "PROACTIVE_SCORING", "DAYS_LAST_INFO_CHANGE"]
df_clients = df_clients.fillna(-1, subset=cols_flag)

# 2. GRUPO "SIN HISTORIAL" (Rellenar con 0)
# El grupo de los 8770 nulos. Si no hay datos de préstamos, asumimos 0.
# Buscamos todas las columnas de préstamos (LOAN_) y estados (NUM_STATUS_)
cols_financieras = [c for c in df_clients.columns if c.startswith("LOAN_") or c.startswith("NUM_")]
# Añadimos otras que tengan sentido ser 0
cols_financieras.extend(["NUM_PREVIOUS_LOAN_APP", "NUMBER_OF_PRODUCTS", "Num_flag_insured"]) # Asegúrate de usar el nombre exacto (mayusc/minusc)

# Filtramos solo las que existen en el DF para no dar error
cols_financieras = [c for c in cols_financieras if c in df_clients.columns]
df_clients = df_clients.fillna(0, subset=cols_financieras)

# 3. GRUPO "CATEGÓRICO DESCONOCIDO" (Rellenar con 'Unknown')
# EDUCATION tiene 39k nulos. No podemos inventárnosla.
cols_categ_nulos = ["EDUCATION", "EMPLOYER_ORGANIZATION_TYPE", "MARITAL_STATUS"]
df_clients = df_clients.fillna("UNKNOWN", subset=cols_categ_nulos)

# 4. GRUPO "ANECDÓTICO" (Estrategia: Salvar al Cliente)
# Al ser poquísimos nulos, preferimos imputar para no perder la ficha del cliente.
# Para Numéricas (Installment): Usamos la MEDIANA (más robusta que la media)
# Calculamos la mediana aproximada (approxQuantile es muy eficiente en Spark)
cols_anecdoticas_num = ["INSTALLMENT", "FAMILY_SIZE"]
for col in cols_anecdoticas_num:
    # Calculamos la mediana de esa columna específica
    mediana = df_clients.stat.approxQuantile(col, [0.5], 0.01)[0]
    df_clients = df_clients.fillna(mediana, subset=[col])

print("Limpieza completada.")

Limpieza completada.


In [25]:
inspeccionar_datos(df_clients, "CLIENTS")


INSPECCIÓN DE: CLIENTS

Dimensiones: 162977 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0--------------------------
 CLIENT_ID                   | 0   
 NON_COMPLIANT_CONTRACT      | 0   
 NAME_PRODUCT_TYPE           | 0   
 GENDER                      | 0   
 TOTAL_INCOME                | 0   
 AMOUNT_PRODUCT              | 0   
 INSTALLMENT                 | 0   
 EDUCATION                   | 0   
 MARITAL_STATUS              | 0   
 HOME_SITUATION              | 0   
 REGION_SCORE                | 0   
 AGE_IN_YEARS                | 0   
 JOB_SENIORITY               | 0   
 HOME_SENIORITY              | 0   
 LAST_UPDATE                 | 0   
 OWN_INSURANCE_CAR           | 0   
 CAR_AGE                     | 0   
 FAMILY_SIZE                 | 0   
 REACTIVE_SCORING            | 0   
 PROACTIVE_SCORING           | 0   
 BEHAVIORAL_SCORING          | 0   
 DAYS_LAST_INFO_CHANGE       | 0   
 NUMBER_OF_PRODUCTS          | 0   
 OCCUPATION                  | 0   
 DIGITAL

In [26]:

# 5. INTEGRACIÓN (JOIN) - MODO NOMBRES ORIGINALES


print(" FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)\n")

col_id = "CLIENT_ID" 

# --- A. SNAPSHOT (Último dato por cliente) ---
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))

# Ventana para quedarnos con el último registro
w = Window.partitionBy(col_id).orderBy(F.desc("DATE"))
df_behav_dedup = df_behav.withColumn("rank", F.row_number().over(w)) \
                         .filter(F.col("rank") == 1) \
                         .drop("rank")

# --- B. JOIN ---
print(f"Uniendo tablas por {col_id}...")
df_master = df_clients.join(df_behav_dedup, on=col_id, how="left")

# --- C. INTEGRIDAD ---
filas_clientes = df_clients.count()
filas_master = df_master.count()

if filas_master > filas_clientes:
    diff = filas_master - filas_clientes
    print(f"AVISO: Aún hay {diff} duplicados.")
else:
    print(f"CORRECTO: 1 Cliente = 1 Fila.")

# --- D. LIMPIEZA POST-JOIN (Nulos a 0) ---
# Tras el join, las columnas financieras de BEHAVIOURAL pueden tener nulos. 
# En el contexto bancario y transaccional, la ausencia de registro significa cantidad cero.
cols_financieras = [
    "CREDICT_CARD_BALANCE", "CREDIT_CARD_LIMIT", "NUMBER_INSTALMENTS", "NUMBER_DRAWINGS", 
    "NUMBER_DRAWINGS_ATM","CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", 
    "CREDIT_CARD_DRAWINGS_OTHER", "CREDIT_CARD_DRAWINGS", "CREDIT_CARD_PAYMENT"
]

cols_existentes = [c for c in cols_financieras if c in df_master.columns]
df_master = df_master.fillna(0, subset=cols_existentes)

# --- E. GESTIÓN DE FECHAS (ESTRATEGIA 'SENTINEL VALUE') ---
# En banca no se borran fechas, se imputa una fecha 'imposible' (1900-01-01).
if "DATE" in df_master.columns:
    print("Aplicando estándar bancario a fechas (Imputación 1900-01-01)...")
    
    # A) Primero calculamos la RECENCIA (Días desde último mov)
    #    Para esto usamos la fecha real antes de "taparla"
    max_date = df_master.agg(F.max("DATE")).collect()[0][0]
    
    df_master = df_master.withColumn(
        "KPI_DAYS_LAST_MOV", 
        F.datediff(F.lit(max_date), F.col("DATE"))
    )
    # Si es nulo (sin datos), ponemos 9999 días (inactivo histórico)
    df_master = df_master.fillna(9999, subset=["KPI_DAYS_LAST_MOV"])
    
    # B) Ahora aplicamos el SENTINEL VALUE a la columna original
    #    Usamos 'coalesce': Si DATE es null, pon 1900-01-01.
    df_master = df_master.withColumn(
        "DATE",
        F.coalesce(F.col("DATE"), F.lit("1900-01-01").cast("date"))
    )

# F. INFORMACIÓN DE LA TABLA 
print(" ESTADO ACTUAL (PRE-REDUCCIÓN)\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print("\nEsquema Actual ")
df_master.printSchema()

 FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)

Uniendo tablas por CLIENT_ID...
CORRECTO: 1 Cliente = 1 Fila.
Aplicando estándar bancario a fechas (Imputación 1900-01-01)...
 ESTADO ACTUAL (PRE-REDUCCIÓN)

Dimensiones: 162977 filas x 59 columnas

Esquema Actual 
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = false)
 |-- EDUCATION: string (nullable = false)
 |-- MARITAL_STATUS: string (nullable = false)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = false)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- C

In [27]:

# 6. FEATURE ENGINEERING Y REDUCCIÓN FINAL

print("FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL\n ")

# --- A. CREACIÓN DE KPIs (Información de Negocio) ---
print("Generando KPIs estratégicos...")

# 1. KPI Gasto Total Tarjeta (Suma de los parciales)
df_master = df_master.withColumn(
    "KPI_TOTAL_SPEND",
    F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER")
)

# 2. KPI Ratio Endeudamiento (Deuda / Ingresos)
# Sumamos 1 al ingreso para evitar divisiones por cero
df_master = df_master.withColumn(
    "KPI_DEBT_RATIO",
    F.round(F.col("CREDICT_CARD_BALANCE") / (F.col("TOTAL_INCOME") + 1), 4)
)

# 3. KPI Grupo de Edad (Simplificación demográfica)
df_master = df_master.withColumn(
    "KPI_AGE_GROUP",
    F.when(F.col("AGE_IN_YEARS") < 30, "Joven")
     .when(F.col("AGE_IN_YEARS") < 50, "Adulto")
     .otherwise("Senior")
)

# 4. KPI Volatilidad de Préstamos (Max - Min)
df_master = df_master.withColumn(
    "KPI_LOAN_VOLATILITY",
    F.col("LOAN_CREDIT_GRANTED_MAX") - F.col("LOAN_CREDIT_GRANTED_MIN")
)

# 5. KPI Ratio de Aprobación (Lo que le dieron / Lo que pidió)
# Indica la confianza del banco en el cliente
df_master = df_master.withColumn(
    "KPI_APPROVAL_RATIO",
    F.round(F.col("LOAN_CREDIT_GRANTED_SUM") / (F.col("LOAN_APPLICATION_AMOUNT_SUM") + 1), 2)
)

# 6. KPI Tasa de Rechazo (Solicitudes denegadas / Total)
# Resume los estados conflictivos
df_master = df_master.withColumn(
    "KPI_DENIAL_RATE",
    F.round(F.col("NUM_STATUS_DENIED") / 
            (F.col("NUM_STATUS_AUTHORIZED") + F.col("NUM_STATUS_DENIED") + F.col("NUM_STATUS_ANNULLED") + 1), 2)
)

# --- B. REDUCCIÓN DE COLUMNAS (Limpieza de "Grasa") ---
print("Eliminando columnas redundantes e inútiles...")

cols_a_borrar = [
    # 1. Redundantes de Tarjeta (Sustituidas por KPI_TOTAL_SPEND)
    "CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", "CREDIT_CARD_DRAWINGS_OTHER", 
    "CREDIT_CARD_DRAWINGS", # Dato duplicado original
    
    # 2. Redundantes de Préstamos (Sustituidas por KPI_LOAN_VOLATILITY y APPROVAL)
    "LOAN_ANNUITY_PAYMENT_MAX", "LOAN_ANNUITY_PAYMENT_MIN",
    "LOAN_APPLICATION_AMOUNT_MAX", "LOAN_APPLICATION_AMOUNT_MIN",
    "LOAN_CREDIT_GRANTED_MAX", "LOAN_CREDIT_GRANTED_MIN",
    "LOAN_VARIABLE_RATE_MAX", "LOAN_VARIABLE_RATE_MIN",

    # 3. Basura Técnica y Duplicados
    "CONTRACT_ID",  # ID interno técnico (inútil para negocio)
    "CURRENCY"      # Columna duplicada por el Join (borramos ambas)
    ]

# Borrado seguro (solo las que existan)
cols_finales_borrar = [c for c in cols_a_borrar if c in df_master.columns]
df_master = df_master.drop(*cols_finales_borrar)

# --- C. DIMENSIONES FINALES ---

print("ESTADO FINAL OPTIMIZADO\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print(f"(Se han eliminado {len(cols_finales_borrar)} columnas redundantes y creado 6 KPIs estratégicos)")

# --- D. GUARDADO FINAL ---
ruta_final = OUTPUT_PATH + "Master_FinPlus.parquet"
df_master.write.mode("overwrite").parquet(ruta_final)
print(f"\nGuardado archivo en: {ruta_final}.")

# --- E. GUARDADO EN PARQUET ÚNICO ---

df_pandas = df_master.toPandas()

nombre_archivo = "Master_FinPlus_Final.parquet"
ruta_completa = OUTPUT_PATH + nombre_archivo

# Necesitas tener instalada la librería pyarrow o fastparquet (suele venir en Docker)
df_pandas.to_parquet(ruta_completa, index=False)

print(f"\nGuardado.")
print(f" Ruta: {ruta_completa}")

df_master.printSchema()
inspeccionar_datos(df_master, "MASTER BOARD")


FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL
 
Generando KPIs estratégicos...
Eliminando columnas redundantes e inútiles...
ESTADO FINAL OPTIMIZADO

Dimensiones: 162977 filas x 50 columnas
(Se han eliminado 14 columnas redundantes y creado 6 KPIs estratégicos)

Guardado archivo en: /home/jovyan/work/data/curated/Master_FinPlus.parquet.

Guardado.
 Ruta: /home/jovyan/work/data/curated/Master_FinPlus_Final.parquet
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = false)
 |-- EDUCATION: string (nullable = false)
 |-- MARITAL_STATUS: string (nullable = false)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = false

In [28]:
def auditar_outliers(df, cols_numericas):
    
    print(f" VERIFICACIÓN DE OUTLIERS (Método IQR)")
    
    
    for col in cols_numericas:
        # 1. Calculamos Cuartiles (25% y 75%)
        quantiles = df.stat.approxQuantile(col, [0.25, 0.75], 0.01)
        q1, q3 = quantiles[0], quantiles[1]
        iqr = q3 - q1
        
        # 2. Definimos límites (Bigotes del Boxplot)
        limite_inf = q1 - 1.5 * iqr
        limite_sup = q3 + 1.5 * iqr
        
        # 3. Contamos cuántos se salen
        outliers = df.filter((F.col(col) < limite_inf) | (F.col(col) > limite_sup))
        num_outliers = outliers.count()
        
        if num_outliers > 0:
            print(f"\n Columna: {col}")
            print(f"   Rango Normal: [{limite_inf:.2f}  a  {limite_sup:.2f}]")
            print(f"   Outliers detectados: {num_outliers} filas")
            
            # Mostramos los valores más extremos para ver si son errores o VIPs
            print(f"   Ejemplos (Top Extremos):")
            outliers.select(col).orderBy(F.desc(col)).show(3)
        else:
            print(f"\n Columna: {col} -> Sin outliers estadísticos.")

# Definimos las columnas numéricas críticas para analizar
cols_analisis = ["TOTAL_INCOME", "AGE_IN_YEARS", "AMOUNT_PRODUCT", "CREDICT_CARD_BALANCE"]

# Ejecutamos
auditar_outliers(df_master, cols_analisis)

 VERIFICACIÓN DE OUTLIERS (Método IQR)

 Columna: TOTAL_INCOME
   Rango Normal: [-270.00  a  4050.00]
   Outliers detectados: 7409 filas
   Ejemplos (Top Extremos):
+------------+
|TOTAL_INCOME|
+------------+
|   1404000.0|
|   216001.08|
|    108000.0|
+------------+
only showing top 3 rows


 Columna: AGE_IN_YEARS -> Sin outliers estadísticos.

 Columna: AMOUNT_PRODUCT
   Rango Normal: [-6455.70  a  19399.50]
   Outliers detectados: 3465 filas
   Ejemplos (Top Extremos):
+--------------+
|AMOUNT_PRODUCT|
+--------------+
|     48486.195|
|     48486.195|
|     48486.195|
+--------------+
only showing top 3 rows


 Columna: CREDICT_CARD_BALANCE
   Rango Normal: [0.00  a  0.00]
   Outliers detectados: 16336 filas
   Ejemplos (Top Extremos):
+--------------------+
|CREDICT_CARD_BALANCE|
+--------------------+
|            14526.13|
|            12122.07|
|            11500.21|
+--------------------+
only showing top 3 rows



### Conversión de variables categóricas a numéricas con Label Encoding

In [29]:
# ==========================================
# 8. TRANSFORMACIÓN FINAL: LABEL ENCODING PARA ML
# ==========================================
print("\n" + "="*40)
print("FASE 4: PREPARACIÓN PARA MODELOS (STRING INDEXER)")
print("="*40)

# 1. Detectar columnas de texto automáticamente
# Definimos la "Lista Negra" de columnas que NO son características (IDs, Fechas, etc.)
cols_excluir = ["CLIENT_ID", "CONTRACT_ID", "DATE", "date"] 

# Buscamos todas las columnas de tipo String que no estén en la lista de exclusión
categ_cols = [f.name for f in df_master.schema.fields 
              if isinstance(f.dataType, T.StringType) and f.name not in cols_excluir]

print(f"Columnas categóricas detectadas para transformar ({len(categ_cols)}):")
print(categ_cols)

# 2. Configurar el StringIndexer (Label Encoding)
# - handleInvalid="keep": Si en el futuro aparece una categoría nueva no vista, crea un índice extra en vez de dar error.
# - outputCol: Crea nuevas columnas terminadas en "_IDX".
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_IDX", handleInvalid="keep") for col in categ_cols]

# 3. Ejecutar la transformación (Pipeline)
# Usamos un Pipeline para aplicar todos los indexers de golpe de forma eficiente
pipeline = Pipeline(stages=indexers)
model = pipeline.fit(df_master)
df_model_ready = model.transform(df_master)

# 4. Selección Final: Solo columnas numéricas
# Nos quedamos con las numéricas originales + los nuevos índices (_IDX).
# Descartamos las columnas de texto originales y los IDs.

# Lista de columnas numéricas originales (Double, Integer, Long)
numeric_cols_orig = [f.name for f in df_master.schema.fields 
                     if isinstance(f.dataType, (T.DoubleType, T.IntegerType, T.LongType))]

# Lista de las nuevas columnas indexadas
idx_cols = [f"{col}_IDX" for col in categ_cols]

# Ensamblamos la lista final
cols_definitivas = numeric_cols_orig + idx_cols
df_final_numeric = df_model_ready.select(cols_definitivas)

# 5. RESULTADO Y GUARDADO
print("DATASET FINAL LISTO PARA ENTRENAMIENTO")

# Mostrar un ejemplo de cómo quedan los datos
print("\nEjemplo de datos transformados:")
df_final_numeric.show(5)

# --- E. GUARDADO EN PARQUET ÚNICO ---
# 1. Convertimos de Spark a Pandas
print("Convirtiendo a formato local (Pandas)...")
df_pandas2 = df_final_numeric.toPandas()

# 2. Definimos nombre y ruta
nombre_archivo = "Master_Model_FinPlus.parquet"
ruta_completa = OUTPUT_PATH + nombre_archivo

# 3. Guardamos el archivo físico
# index=False evita que se guarde el número de fila como una columna extra
df_pandas2.to_parquet(ruta_completa, index=False)

print(f"\n Archivo único guardado en: {ruta_completa}.")

df_final_numeric.printSchema()
inspeccionar_datos(df_final_numeric, "MASTER MODEL")



FASE 4: PREPARACIÓN PARA MODELOS (STRING INDEXER)
Columnas categóricas detectadas para transformar (10):
['NAME_PRODUCT_TYPE', 'GENDER', 'EDUCATION', 'MARITAL_STATUS', 'HOME_SITUATION', 'OWN_INSURANCE_CAR', 'OCCUPATION', 'HOME_OWNER', 'EMPLOYER_ORGANIZATION_TYPE', 'KPI_AGE_GROUP']
DATASET FINAL LISTO PARA ENTRENAMIENTO

Ejemplo de datos transformados:
+----------------------+------------+--------------+-----------+------------------+------------------+-------------+--------------+-----------+-------+-----------+------------------+------------------+------------------+---------------------+------------------+--------------+---------------------+------------------------+---------------------------+-----------------------+-------------------+---------------------+-----------------+-------------------+----------------+--------------------+-----------------+-------------------+-------------------+---------------+------------------+-----------------+---------------+--------------+----------

# INDICADORES Y ANÁLISIS DE COMPORTAMIENTO

## ACTIVIDAD CLIENTE

Este módulo se centra en las métricas de **Recencia, Frecuencia e Intensidad (RFI)** para evaluar el compromiso y la vitalidad de la relación del cliente con los productos de crédito. Es un predictor fundamental del riesgo de abandono y de la lealtad.

El objetivo final es clasificar el nivel de ***engagement*** del cliente en función de cuándo interactuó por última vez, con qué frecuencia lo hace, y **la calidad/volumen de ese uso**.

---

### 1. Métricas Clave

Para este análisis, utilizamos las métricas agregadas del comportamiento transaccional del cliente (`df_behav`), que miden tres dimensiones esenciales del uso del producto:

| Métrica Clave | Definición | Interpretación en Negocio |
| :--- | :--- | :--- |
| **`RECENCY_DAYS`** (R) | Días transcurridos desde la última transacción o actividad del cliente. | Valor bajo ($\le 30$ días) indica que el cliente es activo y **minimiza el riesgo de abandono**. |
| **`FREQUENCY_COUNT`** (F) | Número total de movimientos (gastos y pagos) realizados por el cliente en el período. | Mide la lealtad y el hábito de uso. **Alto Frecuencia = Alto Engagement.** |
| **`INTENSITY_AVG_SPEND`** (I) | Gasto promedio por movimiento. | Mide la **calidad del gasto** o el *ticket* medio del cliente. Es clave para diferenciar el valor real del cliente. |
| **`ACTIVITY_30D/90D/180D`** | Conteo de transacciones en ventanas de tiempo recientes. | Señal temprana de **ralentización o incremento** de la actividad. |

### 2. Lógica de Segmentación

La clasificación (`ACTIVITY_SEGMENT`) ahora utiliza la **Intensidad (I)** para crear un segmento VIP superior, mejorando la precisión de la segmentación. El umbral de Intensidad se establece en la **Mediana ($\text{P50}$)**.

| Segmento | Criterio (Lógica de R, F e I) | Implicación Financiera y Acción Comercial |
| :--- | :--- | :--- |
| **Alta/VIP Intensivo** | **R $\le 30$** y **F $> 5$** **Y** **I $>$ Mediana ($\text{P50}$).** | **Clientes VIP Estratégicos.** Alto *engagement* y alto volumen de negocio. Foco: Productos de Inversión y Alta Fidelización. |
| **Alta** | **R $\le 30$** y **F $> 5$** (sin Intensidad extra). | **Cliente Activo Principal.** Mantiene el uso constante. Objetivo: Retención y Recompensa por uso. |
| **Media** | **R $\le 90$** y **F $> 2$**. | **Cliente Estable/Ocasional.** Potencial de crecimiento. Objetivo: Campañas de Frecuencia (ej., uso semanal). |
| **Baja** | Resto de combinaciones (Inactividad). | **Riesgo de Abandono (Churn).** Baja vitalidad. Objetivo: Reactivación urgente con ofertas de alto impacto. |

In [30]:
# ==========================================
# 1. CARGAR DATOS YA PROCESADOS
# ==========================================
DATA_PATH = "/home/jovyan/work/data/"


beh = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")
df_master = spark.read.parquet(DATA_PATH + "curated/Master_FinPlus.parquet")

beh = beh.withColumn("DATE", F.to_date("DATE", "yyyy-MM-dd"))
max_date = beh.agg(F.max("DATE")).first()[0]

In [31]:
# ==========================================
# 2. MÉTRICAS DE ACTIVIDAD
# ==========================================

# --- RECENCIA ---
recencia_df = beh.groupBy("CLIENT_ID").agg(
    F.max("DATE").alias("last_activity_date")
).withColumn(
    "RECENCY_DAYS", F.datediff(F.lit(max_date), F.col("last_activity_date"))
)

print("\n================ RECENCIA (R) ================\n")
print("Interpretación: número de días desde la última actividad del cliente.")
recencia_df.describe("RECENCY_DAYS").show()

print("\n--- Top 10 clientes más recientes ---")
recencia_df.orderBy(F.col("RECENCY_DAYS").asc()).show(10, truncate=False)

print("\n--- Top 10 clientes más abandonados ---")
recencia_df.orderBy(F.col("RECENCY_DAYS").desc()).show(10, truncate=False)


================ RECENCIA (R) ================

Interpretación: número de días desde la última actividad del cliente.
+-------+------------------+
|summary|      RECENCY_DAYS|
+-------+------------------+
|  count|             46046|
|   mean|15.444794336098685|
| stddev| 21.54619528266414|
|    min|                 0|
|    max|                93|
+-------+------------------+


--- Top 10 clientes más recientes ---
+------------+------------------+------------+
|CLIENT_ID   |last_activity_date|RECENCY_DAYS|
+------------+------------------+------------+
|ES182303796D|2021-12-31        |0           |
|ES182245752Y|2021-12-31        |0           |
|ES182293250V|2021-12-31        |0           |
|ES182245476M|2021-12-31        |0           |
|ES182112694Y|2021-12-31        |0           |
|ES182189508S|2021-12-31        |0           |
|ES182433571C|2021-12-31        |0           |
|ES182232062V|2021-12-31        |0           |
|ES182378189N|2021-12-31        |0           |
|ES182279031S|20

La Recencia mide la salud y la vitalidad del cliente, siendo el predictor más potente del riesgo de abandono (*Churn*).

### 1. Hallazgos Clave

| Métrica | Valor | Implicación |
| :--- | :--- | :--- |
| **Recencia Media** | $\mathbf{15.44}$ días | El cliente promedio ha interactuado en las últimas dos semanas. |
| **Recencia Máxima** | $\mathbf{93}$ días | Define la ventana de análisis. Los clientes en este extremo han cruzado el umbral de $\mathbf{90}$ días de inactividad. |
| **Clientes Activos** | `RECENCY_DAYS` = $\mathbf{0}$ | $\mathbf{2021-12-31}$ es la fecha de corte, confirmando un núcleo de clientes de uso diario. |

### 2. Implicaciones Estratégicas

1.  **Salud del Producto (Alta Vitalidad):**
    * La Recencia Media de $\mathbf{15.44}$ días es muy baja, lo que indica un **alto *engagement* general** y un uso frecuente del producto.

2.  **Riesgo de Abandono (Foco Principal):**
    * Los clientes con $\mathbf{93}$ días de Recencia (actividad en $\text{2021-09-29}$) representan el **Alto Riesgo de Abandono (Churn)**.
    * **Acción:** Este grupo requiere la **máxima prioridad del equipo de Retención** con estrategias de alto impacto (ej., ofertas de reactivación exclusivas).

3.  **Segmentación por Compromiso:**
    * Clientes con Recencia $\mathbf{< 30}$ días (Bajo Riesgo) son el objetivo ideal para el *Up-selling* y programas de fidelización.

In [32]:

# --- FRECUENCIA ---
frecuencia_df = beh.groupBy("CLIENT_ID").agg(
    F.count("*").alias("FREQUENCY_COUNT")
)

print("\n================ FRECUENCIA (F) ================\n")
print("Interpretación: cuántos movimientos totales ha realizado el cliente.")
frecuencia_df.describe("FREQUENCY_COUNT").show()

print("\n--- Top 10 clientes con mayor frecuencia ---")
frecuencia_df.orderBy(F.col("FREQUENCY_COUNT").desc()).show(10, truncate=False)


================ FRECUENCIA (F) ================

Interpretación: cuántos movimientos totales ha realizado el cliente.
+-------+-----------------+
|summary|  FREQUENCY_COUNT|
+-------+-----------------+
|  count|            46046|
|   mean|37.45936672023628|
| stddev|33.78619079695795|
|    min|                1|
|    max|              192|
+-------+-----------------+


--- Top 10 clientes con mayor frecuencia ---
+------------+---------------+
|CLIENT_ID   |FREQUENCY_COUNT|
+------------+---------------+
|ES182186401T|192            |
|ES182128827G|129            |
|ES182192917N|126            |
|ES182283225D|122            |
|ES182378495D|122            |
|ES182155668A|121            |
|ES182253915Y|121            |
|ES182146380G|121            |
|ES182267366X|120            |
|ES182210848C|120            |
+------------+---------------+
only showing top 10 rows



La Frecuencia mide el hábito de uso y la lealtad del cliente (cuántas interacciones totales ha realizado). Es un indicador directo del **engagement constante**.

### 1. Hallazgos Clave

| Métrica | Valor | Implicación |
| :--- | :--- | :--- |
| **Frecuencia Media** | $\mathbf{37.46}$ movimientos | El cliente promedio realiza una transacción cada $\approx 2.5$ días (en el periodo de $\text{93}$ días). |
| **Frecuencia Máxima** | $\mathbf{192}$ movimientos | El cliente más activo realiza casi $\text{2}$ movimientos por día (el cliente es altamente dependiente del producto). |
| **Desviación Estándar** | $\mathbf{33.79}$ | La dispersión es alta, indicando una mezcla de clientes habituales (cercanos a la media) y un segmento de **super-usuarios** (cercanos al máximo). |

### 2. Implicaciones Estratégicas

1.  **Lealtad y Hábito (Muy Alta):**
    * La media de $\mathbf{37.46}$ movimientos en un periodo corto (93 días) sugiere que el producto es una **herramienta financiera principal** para la mayoría de los usuarios.

2.  **Segmento de Super-Usuarios:**
    * El Top 10 de clientes con $\mathbf{120}$ a $\mathbf{192}$ movimientos son los **clientes más leales y dependientes**.
    * **Acción:** Este grupo es perfecto para **pruebas beta de nuevos productos, servicios premium y referidos**, ya que tienen el mayor hábito de uso.

3.  **Oportunidad de Crecimiento (Frecuencia Baja):**
    * Los clientes con Frecuencia cercana a $\text{1}$ o $\text{2}$ movimientos representan una **oportunidad de activación**.
    * **Acción:** Se deben crear campañas que incentiven la **recurrencia** (ej., descuentos por uso semanal o mensual), moviendo a los clientes de baja frecuencia a la media.

In [33]:
# --- INTENSIDAD ---
beh = beh.withColumn(
    "KPI_TOTAL_SPEND",
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_ATM"), F.lit(0)) +
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_POS"), F.lit(0)) +
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_OTHER"), F.lit(0))
)

intensidad_df = beh.groupBy("CLIENT_ID").agg(
    F.avg("KPI_TOTAL_SPEND").alias("INTENSITY_AVG_SPEND")
)

print("\n================ INTENSIDAD (I) ================\n")
print("Interpretación: gasto promedio por movimiento del cliente.")
intensidad_df.describe("INTENSITY_AVG_SPEND").show()

print("\n--- Top 10 clientes con mayor intensidad ---")
intensidad_df.orderBy(F.col("INTENSITY_AVG_SPEND").desc()).show(10, truncate=False)


================ INTENSIDAD (I) ================

Interpretación: gasto promedio por movimiento del cliente.
+-------+-------------------+
|summary|INTENSITY_AVG_SPEND|
+-------+-------------------+
|  count|              46046|
|   mean| 165.34456214658363|
| stddev|  310.5056279798483|
|    min|                0.0|
|    max|  9629.483333333334|
+-------+-------------------+


--- Top 10 clientes con mayor intensidad ---
+------------+-------------------+
|CLIENT_ID   |INTENSITY_AVG_SPEND|
+------------+-------------------+
|ES182227107Z|9629.483333333334  |
|ES182354235A|8642.13            |
|ES182436756T|7395.070000000001  |
|ES182156500S|5901.773846153846  |
|ES182366160A|5899.592142857144  |
|ES182390308Q|5731.825           |
|ES182247132U|5427.0             |
|ES182396618L|5346.0             |
|ES182129030R|5245.332173913043  |
|ES182250483A|5186.4158333333335 |
+------------+-------------------+
only showing top 10 rows



La Intensidad (`INTENSITY_AVG_SPEND`) mide el gasto promedio por movimiento del cliente, siendo un indicador clave de la **calidad del gasto** o el *ticket* medio de las transacciones.

### 1. Hallazgos Clave

| Métrica | Valor | Implicación |
| :--- | :--- | :--- |
| **Intensidad Media** | $\mathbf{165.34€}$ | El gasto promedio por transacción es relativamente bajo, lo que sugiere un **uso habitual para compras diarias** o de valor moderado. |
| **Intensidad Máxima** | $\mathbf{9,629.48€}$ | Existe un segmento de **clientes VIP de alto valor** con transacciones medias extremadamente altas. |
| **Desviación Estándar** | $\mathbf{310.51€}$ | La desviación es casi el doble de la media, lo que indica una **distribución altamente sesgada**. La media no representa al cliente "típico"; está influenciada por los VIP de alto gasto. |

### 2. Implicaciones Estratégicas

1.  **Segmento de Valor Oculto (VIPs):**
    * Los clientes del Top 10, con tickets promedio superiores a $\mathbf{5,000€}$, representan el verdadero **Valor Económico Estratégico** de la base de datos.
    * **Acción:** Estos VIPs deben ser gestionados por un **equipo especializado** (Banca Privada o Servicios Premium) y son los candidatos ideales para productos de inversión de alto *ticket*. 

2.  **Riesgo y Auditoría (Outliers):**
    * La brecha tan grande entre la media ($\text{165€}$) y el máximo ($\text{9,629€}$) resalta la necesidad de implementar el **Módulo de Anomalía Transaccional** (como se verá más adelante).
   
3.  **Masificación del Producto:**
    * El producto está siendo utilizado por la mayoría para transacciones pequeñas/medianas.
    * **Acción:** Para aumentar el *ticket* medio en la base general, se deben lanzar campañas que incentiven la **consolidación de gastos** o el uso de la tarjeta para **compras grandes** (ej., viajes, tecnología).

In [34]:

# ==========================================
# 3. ACTIVIDAD 30 / 90 / 180 DÍAS
# ==========================================
beh_windows = beh.withColumn(
    "DAYS_FROM_REF", F.datediff(F.lit(max_date), F.col("DATE"))
)

ventanas_df = beh_windows.groupBy("CLIENT_ID").agg(
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 30, 1).otherwise(0)).alias("ACTIVITY_30D"),
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 90, 1).otherwise(0)).alias("ACTIVITY_90D"),
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 180, 1).otherwise(0)).alias("ACTIVITY_180D")
)

print("\n================ ACTIVIDAD EN VENTANAS ================\n")
print("Interpretación: cuántas interacciones ha tenido el cliente en los últimos X días.\n")

print("--- Top 10 actividad últimos 30 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_30D").desc()).show(10, truncate=False)

print("--- Top 10 actividad últimos 90 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_90D").desc()).show(10, truncate=False)

print("--- Top 10 actividad últimos 180 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_180D").desc()).show(10, truncate=False)


================ ACTIVIDAD EN VENTANAS ================

Interpretación: cuántas interacciones ha tenido el cliente en los últimos X días.

--- Top 10 actividad últimos 30 días ---
+------------+------------+------------+-------------+
|CLIENT_ID   |ACTIVITY_30D|ACTIVITY_90D|ACTIVITY_180D|
+------------+------------+------------+-------------+
|ES182403907W|2           |6           |12           |
|ES182334710B|2           |6           |12           |
|ES182210848C|2           |6           |12           |
|ES182347896Q|2           |6           |12           |
|ES182243027C|2           |6           |11           |
|ES182407401Q|2           |6           |12           |
|ES182127891W|2           |5           |8            |
|ES182150696Z|2           |6           |12           |
|ES182100594F|2           |4           |7            |
|ES182283225D|2           |6           |12           |
+------------+------------+------------+-------------+
only showing top 10 rows

--- Top 10 actividad ú

Este análisis mide la **recurrencia y el *momentum*** del uso del producto, contando el número de interacciones en ventanas de tiempo específicas. Un crecimiento constante en las ventanas indica una fidelidad estable.

### 1. Hallazgos Clave (Top Recurrentes)

| Métrica | Patrón General (Mayoría del Top 10) | Implicación |
| :--- | :--- | :--- |
| **Actividad 30D** | $\mathbf{2}$ interacciones/mes | El cliente recurrente realiza $\approx 2$ movimientos en el último mes. |
| **Actividad 90D** | $\mathbf{6}$ interacciones/trimestre | El cliente recurrente realiza $\approx 6$ movimientos en el último trimestre. |
| **Actividad 180D** | $\mathbf{12}$ interacciones/semestre | El cliente recurrente realiza $\approx 12$ movimientos en los últimos seis meses. |

### 2. Implicaciones Estratégicas

1.  **Patrón de Uso Estable (Frecuencia Mensual):**
    * La relación $\text{2/6/12}$ (2 movimientos en 30 días, 6 en 90 días, 12 en 180 días) sugiere un patrón de **uso mensual o bimensual constante**.
    * **Causalidad:** Esto indica que el cliente más activo usa el producto de crédito **una o dos veces al mes**, probablemente para pagar una factura o realizar una compra planificada, más que para el gasto diario.

2.  **Detección de Caída de *Engagement* (Alerta Temprana):**
    * Los clientes donde la proporción se rompe (ej., `ACTIVITY_90D` es $\text{4}$ y `ACTIVITY_30D` es $\text{2}$, como el cliente `ES182100594F`) están mostrando una **desaceleración reciente** en la actividad.
    * **Acción:** Estos clientes que no cumplen la proporción de $\text{2/6/12}$ son los candidatos primarios para el **Módulo de Riesgo Potencial (R-Score)**, ya que muestran la caída del *engagement* antes de alcanzar el umbral de $\text{90}$ días de inactividad. 

In [35]:

# ==========================================
# 4. MESES ACTIVOS
# ==========================================
beh_month = beh.withColumn(
    "YEAR_MONTH", F.date_format("DATE", "yyyy-MM")
)

meses_activos_df = beh_month.groupBy("CLIENT_ID").agg(
    F.countDistinct("YEAR_MONTH").alias("ACTIVE_MONTHS")
)

print("\n================ MESES ACTIVOS ================\n")
meses_activos_df.orderBy(F.col("ACTIVE_MONTHS").desc()).show(10, truncate=False)


================ MESES ACTIVOS ================

+------------+-------------+
|CLIENT_ID   |ACTIVE_MONTHS|
+------------+-------------+
|ES182123143P|96           |
|ES182173222Z|96           |
|ES182230930K|96           |
|ES182167286W|96           |
|ES182243311L|96           |
|ES182233737B|96           |
|ES182168740A|96           |
|ES182258895P|96           |
|ES182319130A|96           |
|ES182350305O|96           |
+------------+-------------+
only showing top 10 rows



Este indicador mide la **duración de la relación del cliente** (lealtad histórica) al contar el número de meses en que ha habido actividad transaccional.

### 1. Hallazgos Clave (Top Lealtad)

| Métrica | Valor | Implicación |
| :--- | :--- | :--- |
| **Máximo Histórico** | $\mathbf{96}$ meses | Este es el **límite de tiempo del *dataset*** ($\approx 8$ años). |
| **Top 10 Clientes** | $\mathbf{96}$ meses activos | El Top 10 representa el segmento **más estable y fiel** de la base de datos. |

### 2. Implicaciones Estratégicas

1.  **Lealtad Máxima (VIP Histórico):**
    * Los clientes con $\mathbf{96}$ meses activos son la columna vertebral de la base de clientes. Han estado con el producto desde el inicio del registro histórico y su riesgo de abandono intrínseco es extremadamente bajo.

2.  **Estrategia de Retención:**
    * Las acciones con este grupo no deben centrarse en la activación (ya están activos), sino en el reconocimiento y la retención emocional.
    * **Acción:** Creación de un **programa de lealtad de nivel "Legacy" o "Socio Fundador"** que ofrezca beneficios inigualables para asegurar la continuidad de esta relación a largo plazo.

3.  **Análisis de Deserción:**
    * Este indicador sirve como base: Si un cliente con un historial muy alto (ej., $\text{80}$ meses) pasa al segmento de Riesgo Alto (R-Score), su deserción tiene un coste mucho mayor para la empresa que la deserción de un cliente nuevo.

In [36]:

# ==========================================
# 5. COMBINAR TODAS LAS MÉTRICAS
# ==========================================
activity_metrics = (
    recencia_df
    .join(frecuencia_df, "CLIENT_ID", "left")
    .join(intensidad_df, "CLIENT_ID", "left")
    .join(ventanas_df, "CLIENT_ID", "left")
    .join(meses_activos_df, "CLIENT_ID", "left")
)

In [37]:

# ==========================================
# 6. SEGMENTACIÓN DE ACTIVIDAD (RFI MEJORADO)
# ==========================================

# 1. Calcular el umbral de Intensidad (usamos la mediana o P50 para el 50% superior)
# Esto es necesario para la mejora estratégica.
intensity_threshold = activity_metrics.approxQuantile("INTENSITY_AVG_SPEND", [0.5], 0.01)[0]
print(f"Umbral de Intensidad para VIP (Mediana): {intensity_threshold:.2f}")


# 2. Aplicar la Segmentación RFI Compuesta
activity_metrics = activity_metrics.withColumn(
    "ACTIVITY_SEGMENT",
    # 1. ALTA/VIP INTENSIVO: Reciente (R<=30), Alta Frecuencia (F>5) Y Alta Intensidad (> Mediana)
    F.when(
        (F.col("RECENCY_DAYS") <= 30) & 
        (F.col("FREQUENCY_COUNT") > 5) &
        (F.col("INTENSITY_AVG_SPEND") > intensity_threshold), 
        "Alta/VIP Intensivo"
    )
    # 2. ALTA (Normal): Cumple R y F estándar (sin la intensidad extra)
    .when((F.col("RECENCY_DAYS") <= 30) & (F.col("FREQUENCY_COUNT") > 5), "Alta")
    
    # 3. MEDIA: Reciente o Frecuencia Media
    .when((F.col("RECENCY_DAYS") <= 90) & (F.col("FREQUENCY_COUNT") > 2), "Media")
    
    # 4. BAJA: Resto de casos
    .otherwise("Baja")
)


print("\n================ SEGMENTACIÓN FINAL ================\n")
activity_metrics.groupBy("ACTIVITY_SEGMENT").count().show()

print("\n--- Muestra de clientes segmentados ---\n")
activity_metrics.select(
    "CLIENT_ID", "RECENCY_DAYS", "FREQUENCY_COUNT", 
    "INTENSITY_AVG_SPEND", "ACTIVITY_SEGMENT"
).orderBy(F.col("ACTIVITY_SEGMENT").desc()).show(20, truncate=False)

Umbral de Intensidad para VIP (Mediana): 40.47

================ SEGMENTACIÓN FINAL ================

+------------------+-----+
|  ACTIVITY_SEGMENT|count|
+------------------+-----+
|              Alta|10751|
|             Media|18448|
|              Baja| 1596|
|Alta/VIP Intensivo|15251|
+------------------+-----+


--- Muestra de clientes segmentados ---

+------------+------------+---------------+-------------------+----------------+
|CLIENT_ID   |RECENCY_DAYS|FREQUENCY_COUNT|INTENSITY_AVG_SPEND|ACTIVITY_SEGMENT|
+------------+------------+---------------+-------------------+----------------+
|ES182189657V|31          |7              |0.0                |Media           |
|ES182379297G|62          |36             |0.0                |Media           |
|ES182391573W|31          |95             |0.0                |Media           |
|ES182226833C|0           |4              |39.15              |Media           |
|ES182135783L|62          |5              |0.0                |Media    

Este módulo clasifica la base de clientes según la **Recencia, Frecuencia e Intensidad (RFI)**, identificando los segmentos de mayor y menor *engagement*. El **Umbral VIP** (Intensidad Media) se establece en $\mathbf{40.47€}$.

### 1. Hallazgos Clave y Distribución

| Segmento | Conteo | % Base | Definición Estratégica |
| :--- | :--- | :--- | :--- |
| **Alta/VIP Intensivo** | $\mathbf{15,251}$ | $\mathbf{33.1\%}$ | Clientes Activos, Frecuentes y con **Alto Ticket Medio** ($> 40.47€$). |
| **Media** | $\mathbf{18,448}$ | $\mathbf{40.1\%}$ | El segmento más grande. Clientes con buen uso, pero que fallan en Recencia ($\ge 31$ días) o en la Intensidad. |
| **Alta** | $\mathbf{10,751}$ | $\mathbf{23.3\%}$ | Clientes Activos y Frecuentes, pero que usan el producto para **gastos de bajo valor**. |
| **Baja** | $\mathbf{1,596}$ | $\mathbf{3.5\%}$ | Clientes Inactivos, Durmientes o de nula frecuencia. |

### 2. Implicaciones Estratégicas

1.  **Foco en el VIP (33.1% de la Base):**
    * Más de la tercera parte de los clientes son **VIPs Intensivos** (Recientes, Frecuentes y Alto Gasto por evento).
    * **Acción:** Maximizar la rentabilidad ofreciendo **productos exclusivos y trato preferente** para blindar su fidelidad.

2.  **Oportunidad Crítica (Segmento Media - 40.1%):**
    * El segmento 'Media' es el más grande. La muestra de datos revela que muchos de estos clientes tienen **Alta Frecuencia** (ej., $\text{95}$ o $\text{41}$ movimientos), pero están en 'Media' porque su `RECENCY_DAYS` es $\mathbf{31}$ (justo fuera del límite de 'Alta') o su `INTENSITY_AVG_SPEND` es $\mathbf{0.0}$.
    * **Acción:** Este segmento es la **mayor oportunidad de crecimiento**. Necesitan campañas para corregir su punto débil: **reactivación urgente** si la Recencia es el problema, o **aumento del ticket medio** si la Intensidad es baja.

3.  **Bajo Riesgo de Abandono Masivo:**
    * El segmento 'Baja' (Alto Riesgo de Abandono) es muy reducido ($\mathbf{3.5\%}$).
    * **Acción:** Confirma que el producto tiene una **alta salud de uso** y que los esfuerzos de retención se pueden enfocar en una base muy específica y controlada.

In [38]:
# ==========================================
# 7. UNIR A MASTER FINAL (Persistencia Aislada)
# ==========================================

# Creamos un DataFrame que contiene el df_master base MÁS las métricas de Actividad.
df_master_activity_result = df_master.join(activity_metrics, "CLIENT_ID", "left")

# Guardar el resultado en un nuevo archivo Parquet específico (Estrategia Aislada)
df_master_activity_result.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FinPlus_Activity.parquet")

print("\nArchivo guardado correctamente en: Master_FinPlus_Activity.parquet")
print(f"Total de columnas del archivo de salida: {len(df_master_activity_result.columns)}\n")


Archivo guardado correctamente en: Master_FinPlus_Activity.parquet
Total de columnas del archivo de salida: 59



## VALOR ECONÓMICO

Este módulo cuantifica la **rentabilidad y la capacidad de gasto del cliente**, utilizando un enfoque estratégico que combina el **Volumen de Gasto** con la **Frecuencia de Uso**. El indicador final, E-Score Compuesto, es un filtro esencial para el diseño de estrategias comerciales.

---

### 1. Métricas Clave Generadas

El código calcula un conjunto de métricas basadas en el gasto (`KPI_TOTAL_SPEND`) y la recurrencia (`NUM_TRANSACTIONS`):

| Métrica Clave | Definición | Interpretación en Negocio |
| :--- | :--- | :--- |
| **`TOTAL_SPEND`** | Suma del gasto total acumulado. | Métrica base de rentabilidad (Volumen). |
| **`NUM_TRANSACTIONS`** | Número total de movimientos (Frecuencia de gasto). | Mide el hábito y la **recurrencia** del cliente. |
| **`AVG_TICKET`** | Gasto promedio por transacción (Ticket Medio). | Mide la intensidad del gasto por evento. |
| **`SPEND_RECURRENCE_RATIO`** | Gasto Total / (Nº Transacciones + 1). | Métrica robusta de valor promedio por evento, penalizando la compra única. |

### 2. Lógica de Segmentación (E-Score Compuesto)

La clasificación (`ECONOMIC_VALUE_CLASS`) se realiza mediante la **combinación lógica de los tertiles de Gasto ($\text{P33/P66}$) y la Mediana de Frecuencia ($\text{P50}$).** Este enfoque prioriza a los clientes que gastan mucho **y** lo hacen a menudo. 

| Segmento | Criterio (Lógica del Código) | Implicación Financiera |
| :--- | :--- | :--- |
| **ALTO VALOR (Consolidado)** | Gasto $\ge \text{P66}$ **Y** Frecuencia $>$ Mediana ($\text{P50}$). | **Clientes VIP estables.** Máxima prioridad para retención y productos de alto valor. |
| **ALTO VALOR (Esporádico)** | Gasto $\ge \text{P66}$ **Y** Frecuencia $\le$ Mediana ($\text{P50}$). | **Clientes VIP de riesgo.** Alto valor potencial, pero bajo *engagement*. Foco: Activación de frecuencia. |
| **VALOR MEDIO/ALTO** | Gasto $\ge \text{P33}$ **Y** Frecuencia $>$ Mediana ($\text{P50}$). | **Clientes en crecimiento.** Uso muy frecuente. Objetivo: Venta cruzada para elevar el volumen de gasto. |
| **VALOR MEDIO** | Gasto $\ge \text{P33}$ (sin alta frecuencia). | **Clientes estables.** Monitoreo y campañas de eficiencia. |
| **BAJO VALOR** | Resto de casos. | **Baja prioridad.** |

In [39]:
# ------------------------------------------------------------
# 1. MÉTRICAS ECONÓMICAS BÁSICAS POR CLIENTE
# ------------------------------------------------------------

econ = beh.groupBy("CLIENT_ID").agg(
    # total gastado, volumen de compras
    F.sum("KPI_TOTAL_SPEND").alias("TOTAL_SPEND"), 
    
    # rentabilidad generada (usamos gasto promedio como proxy de valor por transacción)
    F.avg("KPI_TOTAL_SPEND").alias("AVG_TICKET"), # Ticket medio
    
    # varianza del gasto (usamos stddev como medida de dispersión/volatilidad)
    F.stddev("KPI_TOTAL_SPEND").alias("SPEND_STDDEV"),
    
    # compra máxima
    F.max("KPI_TOTAL_SPEND").alias("MAX_PURCHASE"),
    
    # compra mínima
    F.min("KPI_TOTAL_SPEND").alias("MIN_PURCHASE"),
    
    # numero de compras
    F.count("*").alias("NUM_TRANSACTIONS")
)

print("\n================ 1. MÉTRICAS ECONÓMICAS BÁSICAS ================\n")
print("Métricas: Gasto Total, Ticket Medio, Varianza, Max/Min Compra, Nº Transacciones.")

print("✔ Resumen descriptivo del gasto total (TOTAL_SPEND)")
econ.select("TOTAL_SPEND").describe().show()

print("\n✔ Top 10 clientes con mayor gasto total:")
econ.orderBy(F.col("TOTAL_SPEND").desc()).show(10, truncate=False)

print("\n✔ Ticket medio (Top 10 más altos):")
econ.orderBy(F.col("AVG_TICKET").desc()).show(10, truncate=False)

print("\n✔ Distribución de número de transacciones:")
econ.select("NUM_TRANSACTIONS").describe().show()


================ 1. MÉTRICAS ECONÓMICAS BÁSICAS ================

Métricas: Gasto Total, Ticket Medio, Varianza, Max/Min Compra, Nº Transacciones.
✔ Resumen descriptivo del gasto total (TOTAL_SPEND)
+-------+-----------------+
|summary|      TOTAL_SPEND|
+-------+-----------------+
|  count|            46046|
|   mean|3348.860903227201|
| stddev|5342.655025746742|
|    min|              0.0|
|    max|        192187.19|
+-------+-----------------+


✔ Top 10 clientes con mayor gasto total:
+------------+-----------------+------------------+------------------+------------+------------+----------------+
|CLIENT_ID   |TOTAL_SPEND      |AVG_TICKET        |SPEND_STDDEV      |MAX_PURCHASE|MIN_PURCHASE|NUM_TRANSACTIONS|
+------------+-----------------+------------------+------------------+------------+------------+----------------+
|ES182348429U|192187.19        |5057.557631578948 |1830.513040213463 |8507.65     |186.6       |38              |
|ES182227107Z|144442.25        |9629.483333333334

### 1. Hallazgos Clave

| Métrica | Valor | Implicación |
| :--- | :--- | :--- |
| **Gasto Total Medio** | $\mathbf{3,348.86€}$ | El gasto promedio por cliente es moderado, pero la alta desviación indica una **fuerte disparidad** entre clientes. |
| **Gasto Total Máximo** | $\mathbf{192,187.19€}$ | Muestra la existencia de clientes **Extremadamente VIP (Outliers de Volumen)** que concentran una parte significativa del valor. |
| **Ticket Medio Máximo** | $\mathbf{9,629.48€}$ | El cliente de mayor intensidad tiene un gasto promedio por transacción altísimo. |
| **Transacciones Media** | $\mathbf{37.46}$ movimientos | La frecuencia de uso es alta (casi una vez cada $\text{2.5}$ días), lo que es positivo para el *engagement*. |

### 2. Implicaciones Estratégicas

1.  **Regla de Oro (Concentración de Valor):**
    * La inmensa diferencia entre el `mean` ($\text{3,348€}$) y el `max` ($\text{192,187€}$) de `TOTAL_SPEND` subraya el **Principio de Pareto (80/20)**. La rentabilidad está concentrada en un pequeño grupo de clientes.
    * **Acción:** Los clientes en el Top 10 deben ser considerados **clientes estratégicos de la compañía**, no solo de un producto.

2.  **Diferencia entre VIPs de Volumen y de Intensidad:**
    * **VIPs de Volumen:** Clientes como `ES182348429U` ($\text{192k€}$) gastan mucho a lo largo de $\text{38}$ transacciones. Son VIPs consolidados.
    * **VIPs de Intensidad:** Clientes como `ES182354235A` ($\text{8.6k€}$) solo tienen $\mathbf{1}$ o $\mathbf{2}$ transacciones con tickets muy altos. Son VIPs **Esporádicos** y representan una **Oportunidad Alta** de crecimiento, ya que tienen la capacidad de gasto, pero falta la recurrencia.

3.  **Riesgo:**
    * El cliente `ES182132454N` gasta $\text{84.5k€}$ en $\mathbf{87}$ transacciones, pero con un `AVG_TICKET` de solo $\mathbf{972€}$. Esto sugiere que son clientes **extremadamente dependientes** del producto, pero si su actividad se detiene, el impacto en la empresa es alto.

In [40]:
# ------------------------------------------------------------
# 2. RATIO DE RECURRENCIA ECONÓMICA
# ------------------------------------------------------------

econ = econ.withColumn(
    "SPEND_RECURRENCE_RATIO",
    # Ratio: Gasto Total / (Nº Transacciones + 1). Cuanto más alto, mayor el gasto promedio por evento.
    F.round(F.col("TOTAL_SPEND") / (F.col("NUM_TRANSACTIONS") + 1), 2)
)

print("\n================ 2. RATIO DE RECURRENCIA ECONÓMICA ================\n")
print("Ratio: Gasto Total / (Nº Transacciones + 1).")
econ.select("CLIENT_ID", "TOTAL_SPEND", "NUM_TRANSACTIONS", "SPEND_RECURRENCE_RATIO") \
    .orderBy(F.col("SPEND_RECURRENCE_RATIO").desc()) \
    .show(10, truncate=False)


================ 2. RATIO DE RECURRENCIA ECONÓMICA ================

Ratio: Gasto Total / (Nº Transacciones + 1).
+------------+------------------+----------------+----------------------+
|CLIENT_ID   |TOTAL_SPEND       |NUM_TRANSACTIONS|SPEND_RECURRENCE_RATIO|
+------------+------------------+----------------+----------------------+
|ES182227107Z|144442.25         |15              |9027.64               |
|ES182436756T|51765.490000000005|7               |6470.69               |
|ES182366160A|82594.29000000001 |14              |5506.29               |
|ES182156500S|76723.06          |13              |5480.22               |
|ES182129030R|120642.64         |23              |5026.78               |
|ES182348429U|192187.19         |38              |4927.88               |
|ES182446412N|72112.26          |14              |4807.48               |
|ES182250483A|62236.99          |12              |4787.46               |
|ES182358570I|79456.63          |16              |4673.92              

Este ratio (`TOTAL_SPEND / (Nº Transacciones + 1)`) es la métrica más robusta para medir el **valor promedio generado por cada evento** del cliente. Penaliza la actividad trivial y prioriza el gasto sustancial.

### 1. Hallazgos Clave

| Métrica | Valor Típico del Top 10 | Implicación |
| :--- | :--- | :--- |
| **Ratio Máximo** | $\mathbf{9,027.64€}$ | Alto valor por transacción, incluso considerando la frecuencia. |
| **Volumen vs. Frecuencia** | $\text{144k€}$ en $\mathbf{15}$ transacciones | El Ratio identifica a clientes que concentran un **gasto masivo** en pocos eventos. |

### 2. Implicaciones Estratégicas

1.  **Valor Puro por Interacción:**
    * El Ratio de Recurrencia filtra el ruido. Un alto Ratio implica que la **calidad del gasto** es excelente, independientemente de que el cliente gaste $\text{192k€}$ en $\text{38}$ transacciones o $\text{144k€}$ en $\text{15}$.

2.  **Identificación de Oportunidad Extrema:**
    * El cliente `ES182354235A` logra un Ratio de $\mathbf{4,321€}$ con solo $\mathbf{1}$ transacción y $\text{8.6k€}$ de gasto.
    * **Acción:** Este cliente es el arquetipo de la **Oportunidad Alta** (Valor Económico alto, pero Interacción nula). Se debe priorizar su activación inmediata con ofertas de Retorno al Uso para convertir el valor de un solo evento en valor recurrente.

3.  **Priorización de Cartera:**
    * **Acción:** La lista ordenada por este Ratio es el **orden exacto de prioridad** para que el equipo comercial invierta tiempo y recursos. Los clientes con mayor ratio garantizan el mejor retorno por acción.

In [41]:
# ------------------------------------------------------------
# 3. CLASIFICACIÓN DE VALOR ECONÓMICO (E-SCORE: COMPUESTO)
# ------------------------------------------------------------

# 1. Definir umbrales de Gasto (Tertiles)
# Usaremos los tertiles (33% y 66%) de TOTAL_SPEND para la segmentación.
quantiles = econ.approxQuantile("TOTAL_SPEND", [0.33, 0.66], 0.01)
p33, p66 = quantiles

# 2. Definir umbral de Frecuencia (Punto de Recurrencia)
# Usamos la mediana (P50) de NUM_TRANSACTIONS como umbral de "Alta Frecuencia".
frequency_threshold = econ.approxQuantile("NUM_TRANSACTIONS", [0.5], 0.01)[0]


# 3. CLASIFICACIÓN COMPUESTA (Priorizando Recurrencia)
econ = econ.withColumn(
    "ECONOMIC_VALUE_CLASS",
    
    # 1. ALTO VALOR (CONSOLIDADO - VIP Máximo): Alto Gasto (>=p66) AND Alta Frecuencia (> P50)
    F.when(
        (F.col("TOTAL_SPEND") >= p66) & 
        (F.col("NUM_TRANSACTIONS") > frequency_threshold), 
        "ALTO VALOR (Consolidado)"
    )
    # 2. ALTO VALOR (ESPORÁDICO): Alto Gasto (>=p66) pero Frecuencia Baja/Media.
    # Estos son VIPs, pero con riesgo de abandono o que pagan grandes sumas de vez en cuando.
    .when(F.col("TOTAL_SPEND") >= p66, "ALTO VALOR (Esporádico)")
    
    # 3. VALOR MEDIO/ALTO: Gasto Medio (>=p33) Y Alta Frecuencia (> P50).
    # Estos clientes son estables y tienen alta propensión a migrar a Alto Valor.
    .when(
        (F.col("TOTAL_SPEND") >= p33) & 
        (F.col("NUM_TRANSACTIONS") > frequency_threshold), 
        "VALOR MEDIO/ALTO"
    )
    
    # 4. VALOR MEDIO: Gasto Medio (>=p33).
    .when(F.col("TOTAL_SPEND") >= p33, "VALOR MEDIO")
    
    # 5. BAJO VALOR: Resto
    .otherwise("BAJO VALOR")
)


print("\n================ 3. SEGMENTACIÓN DE VALOR ECONÓMICO (E-SCORE COMPUESTO) ================\n")
print(f"Umbral Gasto Alto (P66): {p66:.2f} | Umbral Frecuencia Alta (P50): {frequency_threshold:.0f} transacciones.")

print("\n✔ Segmentación económica completada:")
econ.groupBy("ECONOMIC_VALUE_CLASS").count().show(truncate=False)


================ 3. SEGMENTACIÓN DE VALOR ECONÓMICO (E-SCORE COMPUESTO) ================

Umbral Gasto Alto (P66): 3075.99 | Umbral Frecuencia Alta (P50): 22 transacciones.

✔ Segmentación económica completada:
+------------------------+-----+
|ECONOMIC_VALUE_CLASS    |count|
+------------------------+-----+
|ALTO VALOR (Consolidado)|9421 |
|ALTO VALOR (Esporádico) |6455 |
|VALOR MEDIO             |7135 |
|VALOR MEDIO/ALTO        |8110 |
|BAJO VALOR              |14925|
+------------------------+-----+



Este módulo implementa el **E-Score Compuesto** creando una matriz de valor que combina el **Volumen de Gasto (Eje Y)** y la **Frecuencia/Recurrencia (Eje X)**. El objetivo es diferenciar a los clientes que gastan mucho de aquellos que gastan mucho y lo hacen constantemente.

### 1. Metodología de Clasificación (Matriz E-Score)

La segmentación se basa en tres umbrales clave (los valores se obtienen de la ejecución del código):

| Umbral | Tipo de Métrica | Lógica en el Código |
| :--- | :--- | :--- |
| **Gasto Alto** | `TOTAL_SPEND` $\ge \text{P66}$ | Clientes en el tercio superior de gasto acumulado. |
| **Gasto Medio** | `TOTAL_SPEND` $\ge \text{P33}$ | Clientes en el tercio medio/superior de gasto. |
| **Alta Frecuencia** | `NUM_TRANSACTIONS` $ > \text{P50}$ | Clientes por encima de la mediana de transacciones. |

### 2. Segmentos Estratégicos (Implicaciones de Negocio)

| Segmento Generado | Criterio de Activación | Foco Estratégico |
| :--- | :--- | :--- |
| **ALTO VALOR (Consolidado)** | Gasto Alto **Y** Alta Frecuencia | **VIP Máximo y Fiel.** No necesitan activación, sino blindaje de la relación. Objetivo: **Productos de Alto Valor / Servicios Premium.** |
| **ALTO VALOR (Esporádico)** | Gasto Alto **PERO** Frecuencia Baja | **Oportunidad Crítica.** Tienen la capacidad de gasto, pero no el hábito. Objetivo: **Activación de Frecuencia / Campañas de Recurrencia.** |
| **VALOR MEDIO/ALTO** | Gasto Medio **Y** Alta Frecuencia | **Alto Potencial de Migración.** Clientes muy leales que solo necesitan un incentivo de volumen. Objetivo: **Cross-selling / Aumento del Límite de Crédito.** |
| **VALOR MEDIO / BAJO VALOR** | Gasto Medio o Bajo | **Inversión de Eficiencia.** Se utilizan para campañas de bajo coste o para mantener el volumen. |

### 3. Conclusión Estratégica

La matriz asegura que los recursos de marketing se inviertan prioritariamente en los clientes **"ALTO VALOR (Esporádico)"** y **"VALOR MEDIO/ALTO"**, ya que estos representan el mayor potencial incremental para moverlos hacia el segmento "ALTO VALOR (Consolidado)" y así maximizar el ROI (Retorno de la Inversión).

In [42]:
# ------------------------------------------------------------
# 4. UNIÓN Y PERSISTENCIA AISLADA
# ------------------------------------------------------------

# Definimos las columnas finales del módulo Económico
cols_to_keep_econ = [
    "CLIENT_ID", "TOTAL_SPEND", "AVG_TICKET", "SPEND_STDDEV", "MAX_PURCHASE", 
    "MIN_PURCHASE", "NUM_TRANSACTIONS", "SPEND_RECURRENCE_RATIO", "ECONOMIC_VALUE_CLASS"
]

# Creamos un DataFrame que contiene el df_master base MÁS las métricas económicas.
df_master_econ_result = df_master.join(
    econ.select(*cols_to_keep_econ), 
    on="CLIENT_ID", 
    how="left"
)

# Guardar el resultado en un nuevo archivo Parquet específico.
OUTPUT_PATH = "/home/jovyan/work/data/curated/"
df_master_econ_result.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FinPlus_Econ.parquet")


print("\n================ 4. UNIÓN Y PERSISTENCIA ================\n")
print("Métricas económicas añadidas y guardadas en Master_FinPlus_Econ.parquet (E-Score Compuesto).")
print(f"Total de columnas del archivo de salida: {len(df_master_econ_result.columns)}\n")


================ 4. UNIÓN Y PERSISTENCIA ================

Métricas económicas añadidas y guardadas en Master_FinPlus_Econ.parquet (E-Score Compuesto).
Total de columnas del archivo de salida: 58



## INTERACCIÓN Y FIDELIDAD

Este módulo se enfoca en la **calidad del uso del producto de crédito**, midiendo si el cliente utiliza diversos canales (Diversidad) y si gestiona su crédito de forma responsable (Fidelidad de Pago). Es un indicador directo de la **penetración** y el **riesgo de crédito operativo**.

---

### 1. Métricas Clave Generadas

El código calcula tres grupos principales de métricas a nivel de cliente (`interaccion_df`):

| Tipo | Métrica Clave | Definición | Interpretación en Negocio |
| :--- | :--- | :--- | :--- |
| **Fidelidad (Clave)** | **`PAYMENT_FIDELITY_RATIO`** | Ratio: Pago Total / (Gasto Total + 1). | Mide el uso responsable. Un valor alto indica que el cliente amortiza la deuda o el gasto por encima de la media. |
| **Diversidad** | **`CHANNEL_DIVERSITY_SCORE`** | Puntuación (0-3) que cuenta cuántos canales de gasto (ATM, POS, Other) ha utilizado el cliente. | Mide la **penetración del producto** en el ecosistema de gasto del cliente.  |
| **Canales** | `CHANNEL_ATM/POS/OTHER_RATIO` | Proporción del gasto en cada canal sobre el Gasto Total. | Identifica los canales preferidos, útil para optimizar la infraestructura y las ofertas localizadas. |

### 2. Lógica de Segmentación (I-Score)

La clasificación final (`INTERACTION_SEGMENT`) combina la **Fidelidad** (calidad financiera) y la **Diversidad** (amplitud de uso) para determinar el nivel de *engagement*.

| Segmento | Criterio (Lógica del Código) | Implicación Financiera y Acción Comercial |
| :--- | :--- | :--- |
| **Alta** | **Alta Fidelidad** ($\ge \text{P66}$) **Y** **Alta Diversidad** ($\ge 2$ canales). | **Clientes de Bajo Riesgo Operativo.** Son responsables y usan el producto ampliamente. Objetivo: **Cross-selling** de productos de inversión o seguros. |
| **Media** | **Fidelidad Media** ($\ge \text{P33}$) **O** **Diversidad Media** ($\ge 1$ canal). | **Clientes Estables.** Necesitan un empujón en fidelidad o diversidad. Objetivo: **Activación de Canales** (ej., usar POS si solo usan ATM). |
| **Baja** | Baja Fidelidad y nula Diversidad. | **Clientes de Alto Riesgo Operativo o Durmientes.** Baja penetración. Objetivo: **Campañas de Reactivación** (no financieras, sino de uso). |

In [43]:
# ==========================================
# 0. PREPARACIÓN DE DATOS BASE
# ==========================================

# Rellenar Nulos en las columnas de gasto y pago.
beh = beh.fillna(0, subset=["CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", "CREDIT_CARD_DRAWINGS_OTHER","CREDIT_CARD_PAYMENT", "CREDIT_CARD_DRAWINGS", "NUMBER_DRAWINGS"])

1. Métricas de Uso de Canales y Fidelidad Transaccional

1.1. Cálculo de Gasto Total y Agregación

Se calcula el gasto total por transacción (KPI_TOTAL_SPEND) y luego se agregan todas las transacciones por cliente para obtener los totales (sumas) de gasto por canal y de pagos.

In [44]:
# --- 1.1. KPI de Gasto Total (Total Drawings) por Transacción ---
if "KPI_TOTAL_SPEND" not in beh.columns:
    beh = beh.withColumn("KPI_TOTAL_SPEND", F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER"))

# --- 1.2. Métrica de Repuesta/Fidelidad por CLIENTE (Agregación) ---
interaccion_df = beh.groupBy("CLIENT_ID").agg(
    F.sum("CREDIT_CARD_DRAWINGS_ATM").alias("SPEND_ATM_SUM"),
    F.sum("CREDIT_CARD_DRAWINGS_POS").alias("SPEND_POS_SUM"),
    F.sum("CREDIT_CARD_DRAWINGS_OTHER").alias("SPEND_OTHER_SUM"),
    F.sum("KPI_TOTAL_SPEND").alias("TOTAL_DRAWINGS_SPEND"),
    F.sum("CREDIT_CARD_PAYMENT").alias("TOTAL_PAYMENTS")
)

1.2. Cálculo de Ratios (Manejo de División por Cero)
Para evitar errores de división por cero (que ocurrían si TOTAL_DRAWINGS_SPEND era 0), se utiliza la técnica de sumar + 1 al denominador en todos los cálculos de ratio.

In [45]:
# Cálculo de Ratios
# NOTA: Se mantiene la técnica de sumar +1 al denominador para evitar división por cero (y NaN/Inf)
interaccion_df = interaccion_df.withColumn(
    # Ratio ATM: Proporción del gasto en ATM sobre el Gasto Total.
    "CHANNEL_ATM_RATIO", F.round(F.col("SPEND_ATM_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Ratio POS: Proporción del gasto en POS sobre el Gasto Total.
    "CHANNEL_POS_RATIO", F.round(F.col("SPEND_POS_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Ratio OTHER: Proporción del gasto en Otros sobre el Gasto Total.
    "CHANNEL_OTHER_RATIO", F.round(F.col("SPEND_OTHER_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Métrica de Fidelidad (Gasto vs. Pago): Mide el uso responsable. Valores altos implican que el cliente paga mucho en relación a su gasto.
    "PAYMENT_FIDELITY_RATIO", F.round(F.col("TOTAL_PAYMENTS") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
)

In [46]:
# --- 1.3. Cálculo de ratios y filtro de ruido ---

from pyspark.sql.functions import when, lit

# Definimos el límite de Capping (ej., 50.0 significa 5000% de pago sobre gasto)
CAPPING_LIMIT = 50.0 

interaccion_df = interaccion_df.withColumn(
    "CHANNEL_ATM_RATIO", F.round(F.col("SPEND_ATM_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    "CHANNEL_POS_RATIO", F.round(F.col("SPEND_POS_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    "CHANNEL_OTHER_RATIO", F.round(F.col("SPEND_OTHER_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Paso 1: Calculamos el ratio RAW
    "PAYMENT_FIDELITY_RAW", 
    F.round(F.col("TOTAL_PAYMENTS") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Paso 2: Aplicamos el Filtro de Ruido (Capping) para eliminar outliers de amortización extrema
    "PAYMENT_FIDELITY_RATIO",
    when(F.col("PAYMENT_FIDELITY_RAW") > CAPPING_LIMIT, lit(CAPPING_LIMIT))
     .otherwise(F.col("PAYMENT_FIDELITY_RAW"))
).drop("PAYMENT_FIDELITY_RAW") # Eliminamos la columna temporal RAW

# --- 1.4. Top 10 Clientes Fieles (y Activos) ---
print("\n✔ Top 10 Clientes con Mayor Fidelidad de Pago (APLICANDO CAPPING):")
# FILTRO CRÍTICO: Excluimos a los clientes cuyo gasto total es 0. 
interaccion_df.filter(F.col("TOTAL_DRAWINGS_SPEND") > 0)\
    .orderBy(F.col("PAYMENT_FIDELITY_RATIO").desc())\
    .show(10, truncate=False)


✔ Top 10 Clientes con Mayor Fidelidad de Pago (APLICANDO CAPPING):
+------------+-------------+-------------+---------------+--------------------+------------------+-----------------+-----------------+-------------------+----------------------+
|CLIENT_ID   |SPEND_ATM_SUM|SPEND_POS_SUM|SPEND_OTHER_SUM|TOTAL_DRAWINGS_SPEND|TOTAL_PAYMENTS    |CHANNEL_ATM_RATIO|CHANNEL_POS_RATIO|CHANNEL_OTHER_RATIO|PAYMENT_FIDELITY_RATIO|
+------------+-------------+-------------+---------------+--------------------+------------------+-----------------+-----------------+-------------------+----------------------+
|ES182164679N|0.0          |0.0          |160.38         |160.38              |18396.249999999996|0.0              |0.0              |0.9938             |50.0                  |
|ES182393047B|54.0         |0.0          |0.0            |54.0                |6131.7000000000035|0.9818           |0.0              |0.0                |50.0                  |
|ES182232609P|0.0          |15.05        |

Se evalúa la **calidad del uso del producto**, midiendo la **Fidelidad de Pago** (riesgo crediticio operativo) y la **Diversidad de Canales** (penetración del producto).

### 1. Metodología de Clasificación (Matriz I-Score)

La matriz combina el comportamiento de pago y el *engagement* con el ecosistema de la marca:

| Umbral | Tipo de Métrica | Lógica en el Código |
| :--- | :--- | :--- |
| **Alta Fidelidad** | `PAYMENT_FIDELITY_RATIO` $\ge \text{P66}$ | Clientes con la mejor relación Pago/Gasto (bajo riesgo). |
| **Media Fidelidad** | `PAYMENT_FIDELITY_RATIO` $\ge \text{P33}$ | Clientes con comportamiento de pago estable. |
| **Alta Diversidad** | `CHANNEL_DIVERSITY_SCORE` $\ge 2$ | Clientes que usan $\mathbf{2}$ o $\mathbf{3}$ canales de gasto (alta penetración). |

### 2. Segmentos Estratégicos (Implicaciones de Negocio)

| Segmento Generado | Criterio de Activación | Foco Estratégico |
| :--- | :--- | :--- |
| **Alta** | Alta Fidelidad **Y** Alta Diversidad | **Clientes de Bajo Riesgo Operativo.** Son la base para **Cross-selling** de productos complejos (ej., seguros). |
| **Media** | Media Fidelidad **O** Media Diversidad | **Clientes de Estabilidad Media.** Tienen potencial en una de las dos áreas (pago o uso). Objetivo: **Activación de Canales Específicos** (ej., promover POS si solo usan ATM). |
| **Baja** | Baja Fidelidad **Y** Baja Diversidad | **Alto Riesgo Operativo o Durmientes.** Requieren auditoría o campañas de reactivación enfocadas en el uso. |

### 3. Conclusión Estratégica

La matriz I-Score permite dirigir los esfuerzos para **aumentar la penetración del producto** en el ecosistema del cliente. El foco debe ser **mover los clientes de Fidelidad Media a Alta** mediante incentivos de uso que mejoren su Diversidad de Canales.

In [47]:
# --- 2.1. Diversidad de Canales (Repetición/Uso) ---
# Contar los tipos de transacciones con gasto en la ventana
beh_diversity = beh.groupBy("CLIENT_ID").agg(
    # Contamos el número de transacciones con gasto en cada canal
    F.sum(F.when(F.col("CREDIT_CARD_DRAWINGS_ATM") > 0, 1).otherwise(0)).alias("NUM_ATM_TXN"),
    F.sum(F.when(F.col("CREDIT_CARD_DRAWINGS_POS") > 0, 1).otherwise(0)).alias("NUM_POS_TXN"),
    F.sum(F.when(F.col("CREDIT_CARD_DRAWINGS_OTHER") > 0, 1).otherwise(0)).alias("NUM_OTHER_TXN")
).withColumn(
    "CHANNEL_DIVERSITY_SCORE",
    # CHANNEL_DIVERSITY_SCORE: Suma 1 punto por cada tipo de canal que tenga al menos una transacción (TXN > 0).
    F.when(F.col("NUM_ATM_TXN") > 0, 1).otherwise(0) +
    F.when(F.col("NUM_POS_TXN") > 0, 1).otherwise(0) +
    F.when(F.col("NUM_OTHER_TXN") > 0, 1).otherwise(0)
).drop("NUM_ATM_TXN", "NUM_POS_TXN", "NUM_OTHER_TXN") # Mantenemos el score final


# Unimos el score al DataFrame de métricas de interacción
interaccion_df = interaccion_df.join(beh_diversity, "CLIENT_ID", "left")

print("\n================ 2. DIVERSIDAD DE CANALES (REPETICIÓN) ================\n")
print("Score: 1 punto por cada tipo de gasto de tarjeta utilizado (ATM, POS, Other).")
interaccion_df.groupBy("CHANNEL_DIVERSITY_SCORE").count().orderBy("CHANNEL_DIVERSITY_SCORE").show()


================ 2. DIVERSIDAD DE CANALES (REPETICIÓN) ================

Score: 1 punto por cada tipo de gasto de tarjeta utilizado (ATM, POS, Other).
+-----------------------+-----+
|CHANNEL_DIVERSITY_SCORE|count|
+-----------------------+-----+
|                      0|14652|
|                      1|15582|
|                      2|14730|
|                      3| 1082|
+-----------------------+-----+



Este indicador mide la **penetración del producto** en el ecosistema del cliente, contando cuántos de los tres canales de gasto principales (ATM, POS, Otros) el cliente ha utilizado.

### 1. Hallazgos Clave y Distribución

| Score | Significado | Conteo | % Base ($\approx$) |
| :--- | :--- | :--- | :--- |
| **Score 0** | Nulo uso de canales | $\mathbf{14,652}$ | $31.8\%$ |
| **Score 1** | Uso de 1 canal | $\mathbf{15,582}$ | $33.8\%$ |
| **Score 2** | Uso de 2 canales | $\mathbf{14,730}$ | $32.0\%$ |
| **Score 3** | Uso de 3 canales | $\mathbf{1,082}$ | $2.3\%$ |

### 2. Implicaciones Estratégicas

1.  **Concentración de Clientes:**
    * Casi dos tercios de la base ($\text{65.6\%}$, Scores 0 y 1) utiliza **uno o ningún canal**. Esto indica una **baja penetración y dependencia del producto**.
    * **Acción:** El foco del *marketing* debe ser el segmento de **Score 1** ($\text{15,582}$ clientes) para incentivarlos a usar un **segundo canal (Score 2)**, lo que aumenta la dependencia y reduce el riesgo de abandono.

2.  **Riesgo y Oportunidad (Score 0):**
    * Los $\mathbf{14,652}$ clientes con Score 0 son clientes que, a pesar de estar en la base, no han usado los canales de gasto o solo han realizado pagos.
    * **Acción:** Este segmento es de **Alto Riesgo** de abandono y debe ser el objetivo de campañas de **Activación de Uso Primario**, ofreciendo incentivos fuertes en su primer gasto.

3.  **Segmento Premium Consolidado (Score 3):**
    * El segmento con Score 3 ($\mathbf{1,082}$ clientes) es el más valioso en términos de penetración.
    * **Acción:** Son el grupo ideal para **lanzamientos de nuevos productos** (ej., métodos de pago innovadores o integraciones digitales) y programas de fidelización avanzados.

In [48]:
# ------------------------------------------------------------
# 3. SEGMENTACIÓN DE INTERACCIÓN (I-SCORE)
# ------------------------------------------------------------

# Definir umbrales de Fidelidad usando percentiles (33% y 66%).
fidelidad_quantiles = interaccion_df.approxQuantile("PAYMENT_FIDELITY_RATIO", [0.33, 0.66], 0.01)
f_p33, f_p66 = fidelidad_quantiles

# Umbral de Diversidad: Usar al menos 2 canales (score >= 2) se considera Alta Diversidad.
diversidad_umbral = 2

interaccion_df = interaccion_df.withColumn(
    "INTERACTION_SEGMENT",
    # Segmento ALTA: Cumple con alta fidelidad (percentil 66) Y alta diversidad (2 o 3 canales).
    F.when((F.col("PAYMENT_FIDELITY_RATIO") >= f_p66) & (F.col("CHANNEL_DIVERSITY_SCORE") >= diversidad_umbral), "Alta")
     # Segmento MEDIA: Cumple con media fidelidad (percentil 33) O ha usado al menos 1 canal.
     .when((F.col("PAYMENT_FIDELITY_RATIO") >= f_p33) | (F.col("CHANNEL_DIVERSITY_SCORE") >= 1), "Media")
     # Segmento BAJA: Resto de clientes que tienen baja fidelidad y/o nula diversidad de canales.
     .otherwise("Baja") 
)

print("\n================ 3. SEGMENTACIÓN FINAL DE INTERACCIÓN (I-SCORE) ================\n")
interaccion_df.groupBy("INTERACTION_SEGMENT").count().show()


================ 3. SEGMENTACIÓN FINAL DE INTERACCIÓN (I-SCORE) ================

+-------------------+-----+
|INTERACTION_SEGMENT|count|
+-------------------+-----+
|               Alta| 5620|
|              Media|26900|
|               Baja|13526|
+-------------------+-----+



Este módulo implementa el **I-Score** utilizando una matriz que combina la **Fidelidad de Pago** (Calidad financiera) y la **Diversidad de Canales** (Penetración del producto).

### 1. Metodología de Clasificación (Matriz I-Score)

La segmentación diferencia el uso responsable y amplio del producto de aquel que es limitado o riesgoso.

| Umbral de Métrica | Lógica de Segmentación | Definición Estratégica |
| :--- | :--- | :--- |
| **Alta Fidelidad** | `PAYMENT_FIDELITY_RATIO` $\ge \text{P66}$ | Clientes en el tercio superior de pago sobre gasto (Bajo Riesgo Operativo). |
| **Alta Diversidad** | `CHANNEL_DIVERSITY_SCORE` $\ge 2$ | Uso activo de $\mathbf{2}$ o $\mathbf{3}$ canales de gasto (Alta Penetración). |

### 2. Segmentos Estratégicos (Implicaciones de Negocio)

La lógica utilizada prioriza los clientes estables con potencial:

| Segmento Generado | Criterio de Activación (Lógica del Código) | Foco Estratégico |
| :--- | :--- | :--- |
| **Alta** | Alta Fidelidad ($\ge \text{P66}$) **Y** Alta Diversidad ($\ge 2$) | **VIP de Calidad y Uso.** Son responsables y usan el producto ampliamente. Objetivo: **Cross-selling** de productos de inversión o seguros. |
| **Media** | Media Fidelidad ($\ge \text{P33}$) **O** Diversidad Mínima ($\ge 1$) | **Clientes Estables o con Potencial.** Necesitan un empujón en fidelidad o diversidad. Objetivo: **Activación de Canales** y mejoras en el comportamiento de pago. |
| **Baja** | Resto (Baja Fidelidad **Y** Nula Diversidad) | **Alto Riesgo Operativo o Durmientes.** Requieren auditoría o campañas de reactivación enfocadas en el uso. |

### 3. Conclusión Estratégica

Este módulo es esencial para la gestión del **Riesgo Operativo** y la **Penetración**. El mayor potencial incremental se encuentra en el segmento **Media**, cuyo objetivo es corregir el factor que les impide subir a **Alta** (ya sea la Fidelidad de Pago o la Diversidad de Canales).

In [49]:
# ------------------------------------------------------------
# 4. UNIÓN Y PERSISTENCIA AISLADA
# ------------------------------------------------------------

# Columnas a añadir
cols_to_keep_interac = ["CLIENT_ID", "CHANNEL_ATM_RATIO", "CHANNEL_POS_RATIO", "CHANNEL_OTHER_RATIO",
                        "PAYMENT_FIDELITY_RATIO", "CHANNEL_DIVERSITY_SCORE", "INTERACTION_SEGMENT"]

df_master_interac_result = df_master.join(
    interaccion_df.select(*cols_to_keep_interac), 
    on="CLIENT_ID", 
    how="left"
)

# Guardar el resultado en un nuevo archivo Parquet específico.
OUTPUT_PATH = "/home/jovyan/work/data/curated/"
df_master_interac_result.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FinPlus_Interac.parquet")


print("\n================ 4. UNIÓN Y PERSISTENCIA ================\n")
print("Métricas de interacción y fidelidad añadidas y guardadas en Master_FinPlus_Interac.parquet (Ratio limpio).")
print(f"Total de columnas del archivo de salida: {len(df_master_interac_result.columns)}\n")



================ 4. UNIÓN Y PERSISTENCIA ================

Métricas de interacción y fidelidad añadidas y guardadas en Master_FinPlus_Interac.parquet (Ratio limpio).
Total de columnas del archivo de salida: 56



## RIESGO POTENCIAL (abandono e inactividad)

Esta sección se centra en evaluar la **salud y el compromiso** del cliente mediante métricas temporales que identifican a aquellos con mayor riesgo de abandono (*Churn*). Utilizamos los pilares del modelo RFM (Recencia, Frecuencia).

### 1. Métricas Clave

Las métricas se calculan utilizando la columna `DATE` del historial transaccional (`beh`) como referencia.

| Métrica | Definición | Interpretación en Riesgo |
| :--- | :--- | :--- |
| **`LAST_TXN_DATE`** | Fecha de la última interacción o transacción registrada. | Punto de partida para el cálculo de la recencia. |
| **`DAYS_SINCE_LAST_TXN`** | **Recencia**. Número de días transcurridos entre la última actividad y la fecha de análisis (`max_date`). | **Riesgo:** Valor alto $\rightarrow$ Mayor riesgo de abandono. |
| **`TOTAL_TXN_COUNT`** | **Frecuencia Bruta**. Número total de transacciones (gastos y pagos) realizadas por el cliente. | **Compromiso:** Valor alto $\rightarrow$ Mayor compromiso. |
| **`SPEND_FREQUENCY_RATIO`** | **Propensión al Gasto**. Proporción de transacciones que fueron gasto sobre el total de transacciones. | **Salud:** Sugiere si el cliente usa el producto activamente para gastar o solo para amortizar deuda. |

### 2. Segmentación de Riesgo

Clasificamos a los clientes en segmentos de riesgo basados en los días de inactividad, que es el indicador más directo de abandono (*Churn*).

| Segmento | Criterio (`INACTIVITY_DAYS`) | Implicación de Negocio |
| :--- | :--- | :--- |
| **Bajo** | $< 30$ días | **Cliente activo.** No requiere intervención de retención. |
| **Medio** | $30-89$ días | **Inactivo Reciente**. El cliente ha reducido su actividad y es candidato a campañas de reactivación. |
| **Alto** | $\ge 90$ días | **Abandono Potencial**. El cliente ha dejado de usar el producto. Requiere una estrategia de recuperación urgente. |

In [50]:
# Definir la fecha de análisis (Max Date)
max_date = beh.agg(F.max("DATE")).first()[0]
MAX_DATE_LIT = F.lit(max_date)

# Calcular KPI_TOTAL_SPEND en el nivel transaccional (beh)
# Es necesario para la métrica de SPEND_FREQUENCY_RATIO y coherente con el módulo económico.
beh = beh.withColumn(
    "KPI_TOTAL_SPEND",
    F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER")
)

In [51]:
# ------------------------------------------------------------
# 1. MÉTRICAS DE RIESGO POTENCIAL (RECENCIA Y FRECUENCIA)
# ------------------------------------------------------------

# Agregamos los datos a nivel de CLIENT_ID.
riesgo_df = beh.groupBy("CLIENT_ID").agg(
    # Recencia: Fecha de la última transacción.
    F.max("DATE").alias("LAST_TXN_DATE"), 

    # Frecuencia Bruta: Número total de movimientos (gastos + pagos).
    F.count("*").alias("TOTAL_TXN_COUNT"),

    # Frecuencia de Gasto: Número de veces que el cliente tuvo un gasto > 0.
    F.sum(F.when(F.col("KPI_TOTAL_SPEND") > 0, 1).otherwise(0)).alias("SPEND_TXN_COUNT")
)


# --- 1.1. Cálculo de Recencia, Inactividad y Ratio de Gasto ---

riesgo_df = riesgo_df.withColumn(
    # 1. DAYS_SINCE_LAST_TXN (Recencia): Días transcurridos desde la última actividad.
    "DAYS_SINCE_LAST_TXN",
    F.datediff(MAX_DATE_LIT, F.col("LAST_TXN_DATE"))
).withColumn(
    # 2. INACTIVITY_DAYS: Usamos la recencia como indicador de inactividad.
    "INACTIVITY_DAYS",
    F.col("DAYS_SINCE_LAST_TXN")
).withColumn(
    # 3. SPEND_FREQUENCY_RATIO: Proporción de las transacciones que fueron de gasto.
    "SPEND_FREQUENCY_RATIO",
    F.round(F.col("SPEND_TXN_COUNT") / F.col("TOTAL_TXN_COUNT"), 4)
)


print("\n================ 1. INDICADORES DE RECENCIA Y ACTIVIDAD ================\n")
riesgo_df.select("DAYS_SINCE_LAST_TXN", "SPEND_FREQUENCY_RATIO").describe().show()


================ 1. INDICADORES DE RECENCIA Y ACTIVIDAD ================

+-------+-------------------+---------------------+
|summary|DAYS_SINCE_LAST_TXN|SPEND_FREQUENCY_RATIO|
+-------+-------------------+---------------------+
|  count|              46046|                46046|
|   mean| 15.444794336098685|  0.24331314989358707|
| stddev|  21.54619528266416|   0.2986459125125185|
|    min|                  0|                  0.0|
|    max|                 93|                  1.0|
+-------+-------------------+---------------------+



Este indicador combina la **Recencia (Días de Inactividad)** y el **Ratio de Frecuencia de Gasto** para evaluar el riesgo de abandono (*churn*) y la salud del *engagement* del cliente.

### 1. Hallazgos Clave

| Métrica | Media ($\text{Mean}$) | Implicación Crítica |
| :--- | :--- | :--- |
| **`DAYS_SINCE_LAST_TXN`** | $\mathbf{15.44}$ días | Riesgo de abandono bajo en promedio, pero la alta $\text{Std Dev}$ ($\mathbf{21.54}$) indica una mezcla peligrosa de clientes activos y clientes semi-inactivos (Riesgo Medio). |
| **`SPEND_FREQUENCY_RATIO`** | $\mathbf{0.24}$ | **Solo el $\mathbf{24\%}$ de las transacciones son de gasto.** El $\mathbf{76\%}$ restante se dedica a pagos, liquidación de deuda o amortización, no a la compra activa. |

### 2. Implicaciones Estratégicas

1.  **Riesgo Intrínseco: Cliente de Pago, No de Gasto:**
    * El Ratio de Frecuencia de Gasto ($\mathbf{0.24}$) es el hallazgo más crítico. Demuestra que el producto se percibe mayoritariamente como una herramienta de **crédito/gestión de deuda**, no como un instrumento de gasto diario.
    * **Riesgo:** Una vez que el cliente paga su deuda, el incentivo para mantener la tarjeta activa (y, por lo tanto, el riesgo de abandono) aumenta drásticamente.

2.  **Enfoque de Mitigación (Aumentar el Ratio):**
    * **Acción Primaria:** La estrategia debe centrarse en **aumentar el Ratio de Gasto/Transacción** hacia la media de $1.0$ (uso exclusivo para gasto).
    * **Tácticas:** Incentivos por uso primario (ej., devolución de efectivo en la $\text{10}^{\text{a}}$ compra) o campañas que promuevan la tarjeta como método de pago habitual.

3.  **Identificación de Alto Riesgo de Abandono:**
    * El máximo de $\mathbf{93}$ días de inactividad define al segmento de **Alto Riesgo/Churn** que requiere intervención inmediata del área de Retención, como ya se había identificado previamente.

In [52]:
# ------------------------------------------------------------
# 2. SEGMENTACIÓN DE RIESGO (BASADO EN INACTIVIDAD)
# ------------------------------------------------------------

# Definición de umbrales: 30 días para Inactivo, 90 días para Abandono (Alto Riesgo).
INACTIVE_THRESHOLD = 30 
CHURN_THRESHOLD = 90    

riesgo_df = riesgo_df.withColumn(
    "RISK_SEGMENT",
    F.when(F.col("INACTIVITY_DAYS") >= CHURN_THRESHOLD, "Alto")
     .when(F.col("INACTIVITY_DAYS") >= INACTIVE_THRESHOLD, "Medio")
     .otherwise("Bajo")
)

print("\n================ 2. SEGMENTACIÓN DE RIESGO DE ABANDONO ================\n")
riesgo_df.groupBy("RISK_SEGMENT").count().show()

print("\n--- Muestra de clientes segmentados por Riesgo ---")
riesgo_df.select("CLIENT_ID", "LAST_TXN_DATE", "DAYS_SINCE_LAST_TXN", "RISK_SEGMENT")\
         .orderBy(F.col("DAYS_SINCE_LAST_TXN").desc()).show(10)


================ 2. SEGMENTACIÓN DE RIESGO DE ABANDONO ================

+------------+-----+
|RISK_SEGMENT|count|
+------------+-----+
|       Medio|17575|
|        Alto|  864|
|        Bajo|27607|
+------------+-----+


--- Muestra de clientes segmentados por Riesgo ---
+------------+-------------+-------------------+------------+
|   CLIENT_ID|LAST_TXN_DATE|DAYS_SINCE_LAST_TXN|RISK_SEGMENT|
+------------+-------------+-------------------+------------+
|ES182406226A|   2021-09-29|                 93|        Alto|
|ES182205817D|   2021-09-29|                 93|        Alto|
|ES182211003Y|   2021-09-29|                 93|        Alto|
|ES182451631D|   2021-09-29|                 93|        Alto|
|ES182406265H|   2021-09-29|                 93|        Alto|
|ES182405527S|   2021-09-29|                 93|        Alto|
|ES182211280N|   2021-09-29|                 93|        Alto|
|ES182179190D|   2021-09-29|                 93|        Alto|
|ES182306422P|   2021-09-29|                

Este módulo clasifica a los clientes según sus días de inactividad (`DAYS_SINCE_LAST_TXN`), un indicador directo del **riesgo de deserción (*Churn*)**.

### 1. Metodología de Clasificación

La segmentación se basa en umbrales de inactividad:

| Segmento | Criterio de Inactividad | Conteo | % Base ($\approx$) |
| :--- | :--- | :--- | :--- |
| **Bajo** | $\mathbf{< 30}$ días | $\mathbf{27,607}$ | $60.0\%$ |
| **Medio** | $\mathbf{30 - 89}$ días | $\mathbf{17,575}$ | $38.2\%$ |
| **Alto** | $\mathbf{\ge 90}$ días | $\mathbf{864}$ | $1.9\%$ |

### 2. Implicaciones Estratégicas

1.  **Salud de la Base (Riesgo Bajo/Controlado):**
    * La inmensa mayoría de los clientes ($\mathbf{60\%}$) se encuentra en riesgo **Bajo** (Activos en los últimos $\text{30}$ días). Esto valida la alta vitalidad de la base de clientes.

2.  **Foco Crítico (Riesgo Alto - $1.9\%$):**
    * El segmento de **Riesgo Alto** es pequeño ($\mathbf{864}$ clientes), pero estos clientes no han realizado ninguna actividad en $\mathbf{90}$ días o más, lo que los convierte en clientes **"Perdidos Potenciales"**.
    * **Acción:** El foco del equipo de Retención debe concentrarse al $\mathbf{100\%}$ en estos $\text{864}$ clientes, ya que una base de riesgo tan pequeña permite una **intervención de alto coste y personalizada** (ej., llamadas directas, ofertas de recuperación de alto valor).

3.  **Monitoreo Proactivo (Riesgo Medio - $38.2\%$):**
    * El segmento de $\mathbf{17,575}$ clientes en Riesgo **Medio** (Inactivos entre $\text{30}$ y $\text{89}$ días) representa la **mayor oportunidad de prevención**. 
    * **Acción:** Este grupo requiere campañas de **"Alerta Temprana"** (ej., correos electrónicos de reactivación con incentivos suaves, notificaciones *push*) para evitar que crucen el umbral de $\text{90}$ días y pasen al segmento de Riesgo Alto.

In [ ]:
# ------------------------------------------------------------
# 3. UNIÓN Y PERSISTENCIA AISLADA
# ------------------------------------------------------------

# Columnas a añadir
cols_to_keep_risk = [
    "CLIENT_ID", "LAST_TXN_DATE", "DAYS_SINCE_LAST_TXN", "TOTAL_TXN_COUNT", 
    "SPEND_FREQUENCY_RATIO", "RISK_SEGMENT"
]

# Creamos un DataFrame que contiene el df_master base MÁS las métricas de Riesgo.
df_master_riesgo_result = df_master.join(
    riesgo_df.select(*cols_to_keep_risk), 
    on="CLIENT_ID", 
    how="left"
)

# Guardar el resultado en un nuevo archivo Parquet específico.
OUTPUT_PATH = "/home/jovyan/work/data/curated/"
df_master_riesgo_result.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FinPlus_Riesgo.parquet")


print("\n================ 3. UNIÓN Y PERSISTENCIA ================\n")
print("Métricas de riesgo potencial añadidas y guardadas en Master_FinPlus_Riesgo.parquet.")
print(f"Total de columnas del archivo de salida: {len(df_master_riesgo_result.columns)}\n")


================ 3. UNIÓN Y PERSISTENCIA ================

Métricas de riesgo potencial añadidas y guardadas en Master_FinPlus_Riesgo.parquet.
Total de columnas del archivo de salida: 55



## OPORTUNIDADES COMERCIALES

El objetivo de este módulo de Oportunidades Comerciales es identificar a los clientes que tienen un alto potencial económico (alto valor), pero que actualmente están infrautilizando nuestros productos o canales (baja interacción). Estos clientes representan la mayor oportunidad de crecimiento.



Este módulo combina el **Valor Económico (E-Score)** con la **Interacción (I-Score)** para identificar a los clientes con mayor potencial de crecimiento. El enfoque se centra en la matriz **"Potencial vs. Penetración"**.

El segmento de mayor interés es el de **Oportunidad Comercial Alta**: Clientes con la Capacidad de Gasto para ser rentables, pero que aún no han sido activados o están usando pocos canales.

### 1. Segmentos Clave

Para este análisis, utilizamos los segmentos ya calculados:

| Segmento Clave | Descripción |
| :--- | :--- |
| **`ECONOMIC_VALUE_CLASS`** | Clasificación del cliente basada en el **Gasto Total** (Alto, Medio, Bajo). |
| **`INTERACTION_SEGMENT`** | Clasificación del cliente basada en la **Fidelidad y Diversidad de Canales** (Alta, Media, Baja). |

### 2. Matriz de Oportunidad

Definimos la oportunidad comercial (potencial de crecimiento) mediante las siguientes reglas:

| Oportunidad | Criterio | Acción Comercial |
| :--- | :--- | :--- |
| **Alta Oportunidad** | **VALOR ALTO** y **INTERACCIÓN BAJA/MEDIA** | **Activación/Cross-selling.** Incentivar el uso de más canales o productos.  |
| **Retención/Fidelidad** | **VALOR ALTO** e **INTERACCIÓN ALTA** | **Recompensa/Fidelización.** Clientes que deben ser mantenidos y premiados. |
| **Inversión Limitada** | **VALOR BAJO** o **VALOR MEDIO** | **Monitoreo/Eficiencia.** Baja prioridad para campañas costosas. |

In [54]:
# ==========================================
# 0. PREPARACIÓN DE DATOS Y GARANTÍA DE COLUMNAS
# ==========================================


# Usamos la opción mergeSchema=true si la ruta es correcta
df_master = spark.read.option("mergeSchema", "true").parquet(OUTPUT_PATH + "Master_FinPlus.parquet")

# 1. Definir columnas de segmentación que podrían estar duplicadas en el master.
COLS_SEGMENTATION = ["ECONOMIC_VALUE_CLASS", "INTERACTION_SEGMENT", "COMMERCIAL_OPPORTUNITY"]
COLS_METRICS = ["TOTAL_SPEND"] # Métrica económica necesaria para el análisis Top 5

# 2. PASO CRÍTICO: ELIMINAR COLUMNAS DUPLICADAS Y DE SEGMENTACIÓN EXISTENTES DEL MASTER
COLS_TO_DROP = COLS_SEGMENTATION + COLS_METRICS
df_master = df_master.drop(*COLS_TO_DROP)


# --- UNIÓN TEMPORAL DE SEGMENTOS CLAVE ---
# 3. Unir el segmento de VALOR al master limpio (INCLUIMOS TOTAL_SPEND)
df_temp = df_master.join(
    econ.select("CLIENT_ID", "ECONOMIC_VALUE_CLASS", "TOTAL_SPEND"), # <--- CORRECCIÓN CLAVE
    on="CLIENT_ID", 
    how="left"
)

# 4. Unir el segmento de INTERACCIÓN al master temporal
df_temp = df_temp.join(
    interaccion_df.select("CLIENT_ID", "INTERACTION_SEGMENT"), 
    on="CLIENT_ID", 
    how="left"
)

# 5. Rellenar nulos de clientes sin actividad transaccional
df_temp = df_temp.fillna("BAJO VALOR", subset=["ECONOMIC_VALUE_CLASS"])
df_temp = df_temp.fillna("Baja", subset=["INTERACTION_SEGMENT"])

In [55]:
# ------------------------------------------------------------
# 1. SEGMENTACIÓN DE OPORTUNIDAD COMERCIAL (MATRIZ)
# ------------------------------------------------------------

print("\n================ 1. CREACIÓN DE MATRIZ DE OPORTUNIDAD ================\n")

# Combinamos los segmentos de Valor Económico y Fidelidad de Interacción.
oportunidad_df = df_temp.withColumn(
    "COMMERCIAL_OPPORTUNITY",
    # OPORTUNIDAD ALTA: Clientes con ALTO VALOR, pero BAJA o MEDIA INTERACCIÓN/DIVERSIDAD.
    F.when((F.col("ECONOMIC_VALUE_CLASS") == "ALTO VALOR") & (F.col("INTERACTION_SEGMENT").isin("Baja", "Media")), "Oportunidad Alta")
     
    # FIDELIDAD/RETENCIÓN: Clientes con ALTO VALOR y ALTA INTERACCIÓN.
    .when((F.col("ECONOMIC_VALUE_CLASS") == "ALTO VALOR") & (F.col("INTERACTION_SEGMENT") == "Alta"), "Retención/Fidelidad")
    
    # RIESGO DE MIGRACIÓN: Clientes de VALOR MEDIO/BAJO con ALTA INTERACCIÓN (podrían estar migrando).
    .when((F.col("ECONOMIC_VALUE_CLASS").isin("VALOR MEDIO", "BAJO VALOR")) & (F.col("INTERACTION_SEGMENT") == "Alta"), "Riesgo de Migración")
    
    # BAJA INVERSIÓN: Resto de combinaciones.
    .otherwise("Baja Inversión")
)


================ 1. CREACIÓN DE MATRIZ DE OPORTUNIDAD ================



In [56]:
# ------------------------------------------------------------
# 2. ANÁLISIS Y RESULTADOS
# ------------------------------------------------------------

print("✔ Distribución de Clientes por Segmento de Oportunidad:")
oportunidad_df.groupBy("COMMERCIAL_OPPORTUNITY").count().orderBy(F.col("count").desc()).show(truncate=False)

print("\n✔ Top 5 Clientes de 'Oportunidad Alta' (Clientes Valiosos Infrautilizados):")
oportunidad_df.filter(F.col("COMMERCIAL_OPPORTUNITY") == "Oportunidad Alta")\
              .select("CLIENT_ID", "TOTAL_INCOME", "TOTAL_SPEND", "ECONOMIC_VALUE_CLASS", "INTERACTION_SEGMENT")\
              .orderBy(F.col("TOTAL_INCOME").desc()).show(5, truncate=False)

✔ Distribución de Clientes por Segmento de Oportunidad:
+----------------------+------+
|COMMERCIAL_OPPORTUNITY|count |
+----------------------+------+
|Baja Inversión        |162629|
|Riesgo de Migración   |348   |
+----------------------+------+


✔ Top 5 Clientes de 'Oportunidad Alta' (Clientes Valiosos Infrautilizados):
+---------+------------+-----------+--------------------+-------------------+
|CLIENT_ID|TOTAL_INCOME|TOTAL_SPEND|ECONOMIC_VALUE_CLASS|INTERACTION_SEGMENT|
+---------+------------+-----------+--------------------+-------------------+
+---------+------------+-----------+--------------------+-------------------+



Este módulo identifica el **potencial incremental** de la base de clientes, comparando su **Valor Económico (E-Score)** con su **Fidelidad/Penetración (I-Score)**.

### 1. Hallazgos Clave y Distribución

| Segmento | Conteo | Implicación Estratégica |
| :--- | :--- | :--- |
| **Baja Inversión** | $\mathbf{162,629}$ | El segmento más grande. Clientes de Bajo o Medio Valor con Interacción Media/Baja. Requieren campañas de bajo coste. |
| **Riesgo de Migración** | $\mathbf{348}$ | Clientes de Valor Medio/Bajo **PERO** con Alta Interacción. Son muy fieles; el riesgo es que migren a otro banco si no se les ofrece más valor. |
| **Oportunidad Alta** | $\mathbf{0}$ | **HALLAZGO CRÍTICO.** El segmento objetivo para el crecimiento no existe. |

### 2. Implicaciones Estratégicas

1.  **Vacío en la Oportunidad de Crecimiento:**
    * El segmento "Oportunidad Alta" (Alto Valor Económico + Baja/Media Interacción) está **vacío**.
    * Esto sugiere que **todos los clientes de Alto Valor ya son Clientes Fieles (Retención/Fidelidad)**. 

2.  **Foco Estratégico en 'Riesgo de Migración' ($\mathbf{348}$ clientes):**
    * Este segmento es el **único vector activo de crecimiento inmediato**. Son clientes muy *engaged* que están preparados para gastar más o consolidar sus finanzas.
    * **Acción:** Dirigir campañas específicas para **aumentar su volumen de gasto total** y moverlos a la categoría 'Alto Valor'.

3.  **Inversión Masiva en 'Baja Inversión':**
    * La inmensa mayoría de la base cae en 'Baja Inversión'.
    * **Acción:** Dada su baja rentabilidad individual, cualquier acción masiva debe ser de **bajo coste operativo** (ej., automatización digital, *email marketing*) para no penalizar el ROI.

In [ ]:
# ------------------------------------------------------------
# 3. UNIÓN FINAL Y PERSISTENCIA (Crear Master Final Segmentado)
# ------------------------------------------------------------

# Nota: Este código asume que 'oportunidad_df' ya está calculado en esta celda.

DATA_PATH = "/home/jovyan/work/data/"
OUTPUT_PATH = "/home/jovyan/work/data/curated/"

# 1. Cargar el Master Base Limpio
df_master_base = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus.parquet") 
COLS_BASE = df_master_base.columns

# 2. Cargar los DataFrames de Métricas Aisladas
# Hacemos la carga robusta con las columnas que necesitamos:
df_econ_metrics = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Econ.parquet").select("CLIENT_ID", "ECONOMIC_VALUE_CLASS", "TOTAL_SPEND", "AVG_TICKET")
df_interac_metrics = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Interac.parquet").select("CLIENT_ID", "INTERACTION_SEGMENT", "PAYMENT_FIDELITY_RATIO")
df_riesgo_metrics = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Riesgo.parquet").select("CLIENT_ID", "RISK_SEGMENT", "DAYS_SINCE_LAST_TXN")


# 3. Ensamblar el DataFrame Final completo (incluyendo la nueva columna COMMERCIAL_OPPORTUNITY)
df_master_final_full = df_master_base.select("CLIENT_ID", *[c for c in COLS_BASE if c != "CLIENT_ID"]) # Aseguramos CLIENT_ID primero

# Unir todas las métricas segmentadas
df_master_final_full = df_master_final_full.join(df_econ_metrics, on="CLIENT_ID", how="left")
df_master_final_full = df_master_final_full.join(df_interac_metrics, on="CLIENT_ID", how="left")
df_master_final_full = df_master_final_full.join(df_riesgo_metrics, on="CLIENT_ID", how="left")

# Añadir la columna de Oportunidad Comercial (calculada en esta celda)
df_master_final_full = df_master_final_full.join(
    oportunidad_df.select("CLIENT_ID", "COMMERCIAL_OPPORTUNITY"), 
    on="CLIENT_ID", 
    how="left"
)

# 4. Guardar el resultado en un NUEVO ARCHIVO final (Master_FINAL_SEGMENTED)
# Esto es la solución final y limpia que evita conflictos con el archivo base original.
df_master_final_full.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FINAL_SEGMENTED.parquet")


print("\n================ 3. GUARDADO FINAL ================\n")
print("El proceso ha finalizado. Todas las métricas están consolidadas y guardadas en Master_FINAL_SEGMENTED.parquet.")
print(f"Total de columnas en el maestro final: {len(df_master_final_full.columns)}\n")


================ 3. GUARDADO FINAL ================

El proceso ha finalizado. Todas las métricas están consolidadas y guardadas en Master_FINAL_SEGMENTED.parquet.
Total de columnas en el maestro final: 58



## ANÁLISIS DE CAUSALIDAD / UPLIFT (para identificar qué ofertas realmente causan más retención)

El Análisis de Uplift, o Modelado de Impacto Causal, tiene como objetivo identificar la **ganancia incremental** que una acción (tratamiento) produce en un cliente, en lugar de predecir solo la probabilidad de un resultado.

En este contexto, queremos ver el **impacto causal del historial de crédito (Préstamos) en el Riesgo de Abandono (Churn)**.

### 1. Variables Clave para el Uplift

Necesitamos definir tres componentes en un único dataset: 

1.  **Tratamiento (T):** La acción que recibió el cliente (haber solicitado un préstamo).
2.  **Resultado (Y):** La métrica de riesgo que queremos influir (Riesgo de Abandono).
3.  **Características (X):** Las variables demográficas y de scoring que explican el resultado.

| Variable | Definición | Origen | Tipo en PySpark |
| :--- | :--- | :--- | :--- |
| **`TREATMENT_GROUP`** | Binario que indica si el cliente tuvo historial de solicitud de préstamo (NUM\_PREVIOUS\_LOAN\_APP > 0). **(Tratamiento)** | `df_master` | `Integer` |
| **`Y_RISK_CHURN`** | Binario: $1$ si el cliente está en **Alto Riesgo** (abandono, `RISK_SEGMENT == 'Alto'`); $0$ en caso contrario. **(Resultado)** | `df_master` | `Integer` |
| **`BEHAVIORAL_SCORING`** | Puntuación de comportamiento. **(Características)** | `df_master` | `Double` |

### 2. Lógica del Tratamiento (T)

Como no tenemos una columna explícita de ofertas, definimos el Tratamiento (`TREATMENT_GROUP`) basándonos en la columna `NUM_PREVIOUS_LOAN_APP` del *master*:

* **Grupo de Tratamiento (T=1):** Clientes que tienen un historial de al menos una solicitud de préstamo (`NUM_PREVIOUS_LOAN_APP > 0`).
* **Grupo de Control (T=0):** Clientes sin historial de solicitud de préstamo (`NUM_PREVIOUS_LOAN_APP == 0`).

In [58]:
# ==========================================
# 0. PREPARACIÓN DE DATOS: CARGA DE DEPENDENCIAS
# ==========================================

# 1. Cargar el Master Base limpio (Contiene variables de clientes como TOTAL_INCOME)
df_master_base = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus.parquet")

# 2. Cargar las métricas de Riesgo y Económicas para obtener T e Y.
# Usamos el master base y unimos las métricas que nos faltan (Riesgo).
# Nota: La columna NUM_PREVIOUS_LOAN_APP y BEHAVIORAL_SCORING ya están en df_master_base.
df_riesgo = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Riesgo.parquet").select("CLIENT_ID", "RISK_SEGMENT")

# 3. Ensamblar el DataFrame base para Uplift
uplift_df = df_master_base.join(df_riesgo, on="CLIENT_ID", how="left")

# Rellenar nulos en RISK_SEGMENT si algún cliente no tenía transacciones
uplift_df = uplift_df.fillna("Desconocido", subset=["RISK_SEGMENT"])

In [59]:
# ------------------------------------------------------------
# 1. DEFINICIÓN DE RESULTADO (Y) Y TRATAMIENTO (T)
# ------------------------------------------------------------

print("\n================ 1. DEFINICIÓN DE VARIABLES CAUSALES ================\n")

# --- A. Resultado (Y): Riesgo de Abandono (Churn) ---
# Y=1 si el cliente está en ALTO RIESGO de abandono (Target para retención).
uplift_df = uplift_df.withColumn(
    "Y_RISK_CHURN",
    F.when(F.col("RISK_SEGMENT") == "Alto", 1).otherwise(0)
)
print(f"Resultado (Y): Clientes con Y_RISK_CHURN = 1 (Alto Riesgo): {uplift_df.filter(F.col('Y_RISK_CHURN') == 1).count()}")


# --- B. Tratamiento (T): Historial de Solicitud de Préstamos (Proxy de Oferta) ---
# T=1 si tuvo al menos una solicitud (NUM_PREVIOUS_LOAN_APP > 0).
uplift_df = uplift_df.withColumn(
    "TREATMENT_GROUP",
    F.when(F.col("NUM_PREVIOUS_LOAN_APP") > 0, 1).otherwise(0)
)
print(f"Tratamiento (T): Clientes en Grupo de Tratamiento (T=1): {uplift_df.filter(F.col('TREATMENT_GROUP') == 1).count()}")


================ 1. DEFINICIÓN DE VARIABLES CAUSALES ================

Resultado (Y): Clientes con Y_RISK_CHURN = 1 (Alto Riesgo): 864
Tratamiento (T): Clientes en Grupo de Tratamiento (T=1): 154207


### 1. Definición de las Variables Clave

| Variable | Segmento | Conteo | Definición Estratégica |
| :--- | :--- | :--- | :--- |
| **Resultado (Y)** | $Y\_RISK\_CHURN = \mathbf{1}$ | $\mathbf{864}$ | Clientes clasificados en **Alto Riesgo de Abandono (Churn)**. Este es el comportamiento que se desea **prevenir** o reducir con el Tratamiento. |
| **Tratamiento (T)** | $TREATMENT\_GROUP = \mathbf{1}$ | $\mathbf{154,207}$ | Clientes con historial de solicitud de préstamo. El Tratamiento es la **acción de negocio** que se evalúa como potencial factor de retención. |

### 2. Implicaciones Estratégicas

1.  **Rareza del Riesgo (Y):**
    * El número de clientes en Alto Riesgo ($\mathbf{864}$) es muy bajo en relación al total, lo que es positivo. El modelo de Uplift se centrará en aislar los factores que hacen que esta pequeña fracción se abandone.

2.  **Universalidad del Tratamiento (T):**
    * La inmensa mayoría de la base ($\mathbf{154,207}$ clientes) ha recibido el Tratamiento (historial de préstamo). Esto sugiere que la interacción con el producto de crédito es una **característica estándar del cliente activo**.
    * **Foco del Modelo:** La clave del análisis no será predecir quién abandonará (eso es un modelo de *churn* tradicional), sino determinar si el **historial de préstamo (T=1) realmente disminuye la probabilidad de Alto Riesgo (Y=1)** en comparación con los clientes que **nunca** solicitaron un préstamo (Grupo de Control, $T=0$).

In [65]:
# ------------------------------------------------------------
# 2. ANÁLISIS DESCRIPTIVO DEL UPLIFT
# ------------------------------------------------------------

print("\n================ 2. ANÁLISIS DESCRIPTIVO (TASAS) ================\n")

churn_rates = uplift_df.groupBy("TREATMENT_GROUP").agg(
    F.avg("Y_RISK_CHURN").alias("CHURN_RATE"),
    F.count("*").alias("Total_Clientes")
).cache() 

churn_rates.orderBy("TREATMENT_GROUP").show(truncate=False)

# Cálculo del Uplift Descriptivo (Tasa de Control - Tasa de Tratamiento)
# Se necesitan los datos colectados para el cálculo directo
churn_rates_collect = churn_rates.collect()

rate_control = churn_rates_collect[0]['CHURN_RATE'] if len(churn_rates_collect) > 0 and churn_rates_collect[0]['TREATMENT_GROUP'] == 0 else 0
rate_treatment = churn_rates_collect[1]['CHURN_RATE'] if len(churn_rates_collect) > 1 and churn_rates_collect[1]['TREATMENT_GROUP'] == 1 else 0

descriptive_uplift = (rate_control - rate_treatment) * 100

print(f"\n--- Resultado Descriptivo ---")
print(f"Uplift Descriptivo (Control - Tratamiento): {round(descriptive_uplift, 2)} puntos porcentuales.")
# 


================ 2. ANÁLISIS DESCRIPTIVO (TASAS) ================

+---------------+---------------------+--------------+
|TREATMENT_GROUP|CHURN_RATE           |Total_Clientes|
+---------------+---------------------+--------------+
|0              |0.0030786773090079817|8770          |
|1              |0.005427769167417822 |154207        |
+---------------+---------------------+--------------+


--- Resultado Descriptivo ---
Uplift Descriptivo (Control - Tratamiento): 0 puntos porcentuales.


Este análisis descriptivo establece la Tasa de Alto Riesgo de Abandono (Y=1) entre los clientes con historial de préstamo (T=1, Tratamiento) y los clientes sin historial (T=0, Control).

### 1. Hallazgos Clave (Tasas de Alto Riesgo)

| Grupo | Tasa de Churn (Alto Riesgo) | Base de Clientes |
| :--- | :--- | :--- |
| **Control (T=0)** | $\mathbf{0.308\%}$ | $\mathbf{8,770}$ |
| **Tratamiento (T=1)** | $\mathbf{0.543\%}$ | $\mathbf{154,207}$ |

### 2. Implicaciones Estratégicas

1.  **Relación Inversa (Mayor Riesgo Asociado al Tratamiento):**
    * La Tasa de Alto Riesgo es significativamente **mayor** para el Grupo de Tratamiento ($\mathbf{0.543\%}$) que para el Grupo de Control ($\mathbf{0.308\%}$).
    * Esto implica que, de forma descriptiva, el historial de préstamos (el Tratamiento) está asociado a un **mayor riesgo de abandono**. El *Uplift* descriptivo es, por lo tanto, **negativo** ($\mathbf{-0.235}$ puntos porcentuales, calculando Tasa Control - Tasa Tratamiento).

2.  **Necesidad de Modelado Causal:**
    * Este resultado contraintuitivo (que el historial de préstamo "aumente" el riesgo de *churn*) refuerza la necesidad del **Análisis de Uplift/Causalidad completo**.
    * **Acción:** El modelo causal es necesario para determinar si el historial de préstamo es la **causa** del mayor riesgo, o si el banco ha estado ofreciendo préstamos a un segmento de clientes que ya son, intrínsecamente, **más propensos al riesgo de abandono** (sesgo de selección).

In [ ]:
# ------------------------------------------------------------
# 3. UNIÓN Y PERSISTENCIA AISLADA (Para el Dataset de Uplift)
# ------------------------------------------------------------

# Seleccionamos las variables clave (T, Y, X) y guardamos en un nuevo archivo
cols_to_keep_uplift = ["CLIENT_ID", "Y_RISK_CHURN", "TREATMENT_GROUP", "TOTAL_INCOME", "BEHAVIORAL_SCORING"]
uplift_model_df = uplift_df.select(*cols_to_keep_uplift)

uplift_model_df.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_UPLIFT_DATASET.parquet")

print("\n================ 3. PERSISTENCIA FINAL ================\n")
print(f"Dataset de Uplift generado y guardado en: Master_UPLIFT_DATASET.parquet.")
print(f"Columnas finales (T, Y, X): {uplift_model_df.columns}\n")


================ 3. PERSISTENCIA FINAL ================

Dataset de Uplift generado y guardado en: Master_UPLIFT_DATASET.parquet.
Columnas finales (T, Y, X): ['CLIENT_ID', 'Y_RISK_CHURN', 'TREATMENT_GROUP', 'TOTAL_INCOME', 'BEHAVIORAL_SCORING']



## ANOMALÍA TRANSACCIONAL (para detectar fraudes o glitches del sistema)

Este módulo se centra en la identificación de transacciones y clientes atípicos que se salen del patrón de gasto habitual. El objetivo es doble: **controlar el fraude** (aumento repentino de gasto) y **auditar la calidad de los datos** (*glitches* del sistema).

### 1. Métricas Clave

Usaremos dos métricas económicas para la detección de anomalías:

| Métrica | Propósito | Umbral (Método IQR) |
| :--- | :--- | :--- |
| **`TOTAL_SPEND`** | **Anomalía de Volumen.** Detecta clientes que han gastado una cantidad total *excesivamente* alta en el período, indicando una posible exposición al fraude o un cliente VIP inusual. | Límite Superior del IQR |
| **`AVG_TICKET`** | **Anomalía de Intensidad.** Detecta clientes cuyo gasto promedio por transacción es *excesivamente* alto. | Límite Superior del IQR |

### 2. Detección por Rango Intercuartílico (IQR)

El método IQR es robusto contra la distribución asimétrica del gasto (que es común en banca).

* **IQR:** $Q3 - Q1$
* **Límite Superior (Outlier):** $Q3 + 1.5 \times IQR$

El cliente se clasifica como **`ANOMALY_FLAG = 1`** si su `TOTAL_SPEND` o su `AVG_TICKET` supera este límite superior.

In [62]:
from pyspark.sql.functions import when, lit

# ------------------------------------------------------------
# 1. CÁLCULO DE UMBRALES DE ANOMALÍA (MÉTODO IQR)
# ------------------------------------------------------------

print("\n================ 1. CÁLCULO DE UMBRALES IQR ================\n")

# --- 1.1. Umbrales para TOTAL_SPEND (Volumen) ---
quantiles_spend = econ.approxQuantile("TOTAL_SPEND", [0.25, 0.75], 0.01)
Q1_spend, Q3_spend = quantiles_spend
IQR_spend = Q3_spend - Q1_spend
LIMITE_SUPERIOR_SPEND = Q3_spend + 1.5 * IQR_spend

print(f"Umbral de Anomalía para TOTAL_SPEND (Límite Superior): {LIMITE_SUPERIOR_SPEND:.2f}")
print(f"Cuartil Superior (Q3) para TOTAL_SPEND (Alto Valor): {Q3_spend:.2f}")


# --- 1.2. Umbrales para AVG_TICKET (Intensidad/Riesgo) ---
econ_temp = econ.withColumn("AVG_TICKET_CLEAN", F.coalesce(F.col("AVG_TICKET"), F.lit(0)))
quantiles_ticket = econ_temp.approxQuantile("AVG_TICKET_CLEAN", [0.25, 0.75], 0.01)
Q1_ticket, Q3_ticket = quantiles_ticket
IQR_ticket = Q3_ticket - Q1_ticket
LIMITE_SUPERIOR_TICKET = Q3_ticket + 1.5 * IQR_ticket

print(f"Umbral de Anomalía para AVG_TICKET (Límite Superior): {LIMITE_SUPERIOR_TICKET:.2f}")


# ------------------------------------------------------------
# 2. DETECCIÓN Y CLASIFICACIÓN DE ANOMALÍAS POR NEGOCIO
# ------------------------------------------------------------

print("\n================ 2. DETECCIÓN Y CLASIFICACIÓN DE ANOMALÍAS ================\n")

anomalia_df = econ.withColumn(
    # Flag binario para volumen extremo (VIP / Potencial Fraude)
    "IS_OUTLIER_SPEND", 
    when(F.col("TOTAL_SPEND") > LIMITE_SUPERIOR_SPEND, 1).otherwise(0)
).withColumn(
    # Flag binario para ticket extremo (Riesgo / Fraude)
    "IS_OUTLIER_TICKET", 
    when(F.col("AVG_TICKET") > LIMITE_SUPERIOR_TICKET, 1).otherwise(0)
)

# Creamos la CLASE DE ANOMALÍA (ANOMALY_CLASS)
anomalia_df = anomalia_df.withColumn(
    "ANOMALY_CLASS",
    # 1. VIP EXTREMO: Outlier de Gasto Total (Es nuestro mayor cliente, necesita auditoría VIP, no eliminación)
    when(F.col("IS_OUTLIER_SPEND") == 1, lit("VIP Extremo / Alto Volumen"))
    
    # 2. RIESGO/ERROR: Outlier de Ticket Medio (Transacción anormalmente grande, riesgo de fraude o error de sistema)
    .when(F.col("IS_OUTLIER_TICKET") == 1, lit("Potencial Fraude / Error"))
    
    # 3. ALTO VALOR (Normal): No es un outlier, pero está en el cuartil superior de gasto.
    .when(F.col("TOTAL_SPEND") >= Q3_spend, lit("Alto Valor Normal")) 
    
    # 4. Normal
    .otherwise(lit("Normal"))
)


# Análisis de la nueva clasificación
print("✔ Conteo de Anomalías y VIPs (ANOMALY_CLASS):")
anomalia_df.groupBy("ANOMALY_CLASS").count().show(truncate=False)

print("\n✔ Top 5 Clientes con Mayor Gasto (Deberían ser 'VIP Extremo'):")
anomalia_df.select("CLIENT_ID", "TOTAL_SPEND", "AVG_TICKET", "ANOMALY_CLASS")\
    .orderBy(F.col("TOTAL_SPEND").desc()).show(5, truncate=False)


# ------------------------------------------------------------
# 3. UNIÓN Y PERSISTENCIA AISLADA
# ------------------------------------------------------------

# Seleccionamos solo las métricas finales de anomalía para la unión
cols_to_keep_anomalia = ["CLIENT_ID", "ANOMALY_CLASS", "IS_OUTLIER_SPEND", "IS_OUTLIER_TICKET"]
anomalia_df_final = anomalia_df.select(*cols_to_keep_anomalia)

# Unimos al df_master base
df_master_anomalia_result = df_master.join(anomalia_df_final, on="CLIENT_ID", how="left")

# Guardar el resultado en un nuevo archivo Parquet específico (Guardado Aislado)
df_master_anomalia_result.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FinPlus_Anomalia.parquet")

print("\n================ 3. PERSISTENCIA FINAL ================\n")
print(f"Métricas de anomalía añadidas y guardadas en Master_FinPlus_Anomalia.parquet.")
print(f"Total de columnas del archivo de salida: {len(df_master_anomalia_result.columns)}\n")


================ 1. CÁLCULO DE UMBRALES IQR ================

Umbral de Anomalía para TOTAL_SPEND (Límite Superior): 10800.00
Cuartil Superior (Q3) para TOTAL_SPEND (Alto Valor): 4320.00
Umbral de Anomalía para AVG_TICKET (Límite Superior): 527.05

================ 2. DETECCIÓN Y CLASIFICACIÓN DE ANOMALÍAS ================

✔ Conteo de Anomalías y VIPs (ANOMALY_CLASS):
+--------------------------+-----+
|ANOMALY_CLASS             |count|
+--------------------------+-----+
|VIP Extremo / Alto Volumen|3137 |
|Potencial Fraude / Error  |1986 |
|Alto Valor Normal         |7232 |
|Normal                    |33691|
+--------------------------+-----+


✔ Top 5 Clientes con Mayor Gasto (Deberían ser 'VIP Extremo'):
+------------+-----------+-----------------+--------------------------+
|CLIENT_ID   |TOTAL_SPEND|AVG_TICKET       |ANOMALY_CLASS             |
+------------+-----------+-----------------+--------------------------+
|ES182348429U|192187.19  |5057.557631578948|VIP Extremo / Alto Vol

Este módulo utiliza el método **IQR (Rango Intercuartílico)** para identificar transacciones y clientes atípicos, separando el **riesgo potencial de fraude o error** del comportamiento de los **clientes VIP de alto volumen**.

### 1. Metodología y Umbrales Clave

La clasificación se basa en la superación de dos límites superiores (Outliers) distintos:

| Métrica | Umbral (Límite Superior IQR) | Interpretación del Outlier |
| :--- | :--- | :--- |
| **Gasto Total** (`TOTAL_SPEND`) | $\mathbf{10,800.00€}$ | **Volumen Extremo** (Clasificado como VIP). |
| **Ticket Medio** (`AVG_TICKET`) | $\mathbf{527.05€}$ | **Intensidad Extrema** (Clasificado como Fraude/Error). |

### 2. Segmentos Estratégicos (Conteo de Outliers)

| Segmento | Conteo | Definición y Acción Estratégica |
| :--- | :--- | :--- |
| **VIP Extremo / Alto Volumen** | $\mathbf{3,137}$ | Clientes con gasto total $\ge \mathbf{10,800€}$. **Acción:** Retención de alto valor y monitoreo de riesgo, pero no eliminación. |
| **Potencial Fraude / Error** | $\mathbf{1,986}$ | Clientes con ticket medio $\ge \mathbf{527.05€}$ (pero que no son ya VIPs de volumen). **Acción:** Prioridad de **Auditoría de Fraude y Sistemas** para validar la legitimidad de sus transacciones. |
| **Alto Valor Normal** | $\mathbf{7,232}$ | Clientes con gasto entre $\mathbf{4,320€}$ ($\text{Q3}$) y $\mathbf{10,800€}$. **Acción:** Clientes de valor estable y bajo riesgo; objetivo de *Up-selling*. |

### 3. Conclusión Estratégica

La segmentación es eficaz al **separar la anomalía de riesgo de la anomalía de valor**. Los $\mathbf{3,137}$ "VIP Extremos" confirman que la mayoría de los *outliers* de volumen son clientes rentables, lo que evita que el sistema de riesgo los penalice indebidamente. El foco de la mitigación de riesgo debe concentrarse específicamente en los $\mathbf{1,986}$ clientes clasificados como "Potencial Fraude / Error".

## CÓDIGO FINAL DE CONSOLIDACIÓN

Se ha estado utilizando la estrategia de Guardado Aislado, donde cada módulo guarda sus resultados en un archivo Parquet independiente (ej., Master_FinPlus_Econ.parquet).

El propósito de la Consolidación Final es unir todos esos archivos de métricas en una única tabla maestra de clientes (el "Maestro Final"), lista para ser usada en modelos predictivos o análisis de negocio.

In [63]:
# ==========================================
# CONSOLIDACIÓN FINAL DE SEGMENTOS
# ==========================================

# 1. Cargar el Master Base Limpio (Contiene Demográficos y KPIs de Feature Engineering)
print("1. Cargando Master Base limpio...")
df_master_base = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus.parquet") 
COLS_BASE = df_master_base.columns


# 2. Cargar todos los DataFrames de Métricas Aisladas
print("2. Cargando métricas segmentadas de los archivos aislados...")

# Módulo Actividad Cliente
df_activity = spark.read.parquet(DATA_PATH + "Master_FinPlus_Activity.parquet").select("CLIENT_ID", "ACTIVITY_SEGMENT", "RECENCY_DAYS", "FREQUENCY_COUNT")

# Módulo Valor Económico
df_econ = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Econ.parquet").select("CLIENT_ID", "ECONOMIC_VALUE_CLASS", "TOTAL_SPEND", "AVG_TICKET")

# Módulo Interacción y Fidelidad
df_interac = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Interac.parquet").select("CLIENT_ID", "INTERACTION_SEGMENT", "PAYMENT_FIDELITY_RATIO", "CHANNEL_DIVERSITY_SCORE")

# Módulo Riesgo Potencial
df_riesgo = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Riesgo.parquet").select("CLIENT_ID", "RISK_SEGMENT", "DAYS_SINCE_LAST_TXN")

# Módulo Uplift/Causalidad
df_uplift = spark.read.parquet(OUTPUT_PATH + "Master_UPLIFT_DATASET.parquet").select("CLIENT_ID", "TREATMENT_GROUP", "Y_RISK_CHURN")

# Módulo Anomalía Transaccional
df_anomalia = spark.read.parquet(OUTPUT_PATH + "Master_FinPlus_Anomalia.parquet").select("CLIENT_ID", "ANOMALY_CLASS")


# 3. Ensamblar el DataFrame Final (JOIN de todas las piezas)
print("3. Ensamblando el Maestro Final...")
df_final_consolidado = df_master_base.select("CLIENT_ID", *[c for c in COLS_BASE if c != "CLIENT_ID"]) 

df_final_consolidado = df_final_consolidado.join(df_activity, on="CLIENT_ID", how="left")
df_final_consolidado = df_final_consolidado.join(df_econ, on="CLIENT_ID", how="left")
df_final_consolidado = df_final_consolidado.join(df_interac, on="CLIENT_ID", how="left")
df_final_consolidado = df_final_consolidado.join(df_riesgo, on="CLIENT_ID", how="left")
df_final_consolidado = df_final_consolidado.join(df_uplift, on="CLIENT_ID", how="left")
df_final_consolidado = df_final_consolidado.join(df_anomalia, on="CLIENT_ID", how="left")


# 4. Recálculo de la Oportunidad Comercial (Última Capa Estratégica)
print("4. Calculando segmento estratégico (COMMERCIAL_OPPORTUNITY)...")
df_final_consolidado = df_final_consolidado.fillna("BAJO VALOR", subset=["ECONOMIC_VALUE_CLASS"])
df_final_consolidado = df_final_consolidado.fillna("Baja", subset=["INTERACTION_SEGMENT"])

df_final_consolidado = df_final_consolidado.withColumn(
    "COMMERCIAL_OPPORTUNITY",
    F.when((F.col("ECONOMIC_VALUE_CLASS") == "ALTO VALOR") & (F.col("INTERACTION_SEGMENT").isin("Baja", "Media")), "Oportunidad Alta")
    .when((F.col("ECONOMIC_VALUE_CLASS") == "ALTO VALOR") & (F.col("INTERACTION_SEGMENT") == "Alta"), "Retención/Fidelidad")
    .when((F.col("ECONOMIC_VALUE_CLASS").isin("VALOR MEDIO", "BAJO VALOR")) & (F.col("INTERACTION_SEGMENT") == "Alta"), "Riesgo de Migración")
    .otherwise("Baja Inversión")
)


# 5. GUARDADO FINAL DEFINITIVO
print("\n================ 5. GUARDADO FINAL DEFINITIVO ================\n")

# Guardamos el maestro final en un nuevo archivo (Máxima estabilidad)
df_final_consolidado.write.mode("overwrite").parquet(OUTPUT_PATH + "Master_FINAL_CONSOLIDADO.parquet")

print("✔ PROCESO FINALIZADO.")
print("El Master Final está consolidado en: Master_FINAL_CONSOLIDADO.parquet.")
print(f"Total de columnas finales (Métricas + Base): {len(df_final_consolidado.columns)}\n")

1. Cargando Master Base limpio...
2. Cargando métricas segmentadas de los archivos aislados...
3. Ensamblando el Maestro Final...
4. Calculando segmento estratégico (COMMERCIAL_OPPORTUNITY)...

================ 5. GUARDADO FINAL DEFINITIVO ================

✔ PROCESO FINALIZADO.
El Master Final está consolidado en: Master_FINAL_CONSOLIDADO.parquet.
Total de columnas finales (Métricas + Base): 65

